## Imports

In [1574]:
import tkinter as tk # Biblioteca Tkinter para fazer a interface.
# Objetos e classes importantes da biblioteca Tkinter para fazer a interface.
from tkinter import ttk, simpledialog, filedialog, messagebox, scrolledtext, Checkbutton, Entry, IntVar, DISABLED, NORMAL
import pandas as pd # Biblioteca Pandas para trabalhar com dataframes e planilhas do Excel.
import subprocess # Biblioteca Subprocess para chamar a execução de scripts/outros códigos em Python.
import os # Biblioteca OS para auxiliar na criação e salvamento de arquivos.
# Biblioteca Openpyxl para trabalhar com planilhas do Excel de forma mais elaborada.
import openpyxl 
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
from openpyxl.formatting.rule import DataBar, FormatObject, Rule
from datetime import datetime, timedelta # Funções para a leitura de horários de aula

# Obsoleto

## Selecionar Arquivo

In [1575]:
def selecionar_arquivo(window_title):
    # Abre o explorador de arquivos para escolher um arquivo
    # print(sheet)
    # if sheet in ["SME", "SMA", "SCC", "SSC"]:
    #     messagebox.showinfo("", f"Selecione a planilha com os dados do {sheet}.")
    # elif sheet == "Salas":
    #     messagebox.showinfo("", f"Selecione a planilha com os dados das salas.")
        
    caminho_arquivo = filedialog.askopenfilename(
        title=window_title,
        filetypes=[("Todos os arquivos", "*.*"),("Arquivos Excel", "*.xlsx *.xls"), ("Arquivos CSV", "*.csv")]
    )
    
    if caminho_arquivo:
        try:
            # Verifica a extensão do arquivo e lê como DataFrame
            if caminho_arquivo.endswith(('.xlsx', '.xls')):
                df = pd.read_excel(caminho_arquivo)
            elif caminho_arquivo.endswith('.csv'):
                df = pd.read_csv(caminho_arquivo, on_bad_lines='skip', sep=';', encoding='latin-1')
            else:
                raise ValueError("Formato de arquivo não suportado!")
            
            # Exibe o DataFrame no console (ou faça outra lógica)
            # print(df.head())
            return df,caminho_arquivo
            # try:
            #     subprocess.run(["python", "Modelo Principal Tempo Contínuo M1.1.py", df, salas])
            messagebox.showinfo("Sucesso", f"Arquivo carregado com sucesso!\n{caminho_arquivo}")
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao carregar o arquivo:\n{e}")
    else:
        messagebox.showwarning("Aviso", "Nenhum arquivo foi selecionado.")
    return None, "Not gud"

## Construção Planilha Grande

In [1576]:
def construcao_planilha_grande():
    sheets = ["SME", "SMA", "SCC", "SSC"]
    # sheets = ["SME", "SMA", "SCC", "SSC", "Outros Departamentos"]
    # sheets = ["SME"]
    files = []
    for sh in sheets:
        messagebox.showinfo("", f"Selecione a planilha com os dados do {sh}.")
        df, file_name = selecionar_arquivo(f"Selecione a planilha com os dados do {sh}.")
        if file_name == "Not gud":
            root.destroy()
            return
        header_name = 'Disciplina (código)'
        for i, valor in enumerate(df.loc[:,df.columns[0]]):
            if valor == header_name:
                print(f"Dataframe do {sh} lido com sucesso.")
                header_row = i+1
                files.append(padroniza_dataframe(file_name, header_row, ano))
                # print(type(files[sh]))
                break
    # Em tese, tenho todas as planilhas no files já
    # Hora de criar o arquivo com todas as planilhas juntas e com a lista de salas
    # Salas = pd.read_excel("Planilha das salas.xlsx")
    messagebox.showinfo("", f"Selecione a planilha com os dados das salas.")
    Salas, file_name = selecionar_arquivo(f"Selecione a planilha com os dados das Salas.")
    
    print(f"Dataframe das Salas lido com sucesso.")

    nome_arquivo = simpledialog.askstring("Input", "Digite o nome do arquivo:")
    if not nome_arquivo.endswith(".xlsx"):
        nome_arquivo += ".xlsx"
        
    with pd.ExcelWriter(nome_arquivo, engine="openpyxl") as writer:
        Salas.to_excel(writer, sheet_name="Salas", index=False)
        for sh in range(len(sheets)):
            files[sh].to_excel(writer, sheet_name=sheets[sh], index=False)
    messagebox.showinfo("", f"Arquivo {nome_arquivo} criado com sucesso!")

## Construção Jupiter

In [1577]:
def construcao_jupiter():
    sheets = ["SME", "SMA", "SCC", "SSC"]
    # sheets = ["SME"]
    files = []
    for sh in sheets:
        messagebox.showinfo("", f"Selecione a planilha com os dados do {sh}.")
        dataframe, file_name = selecionar_arquivo(f"Selecione a planilha com os dados do {sh}.")
        
        if file_name == "Not gud":
            root.destroy()
            return
        
        files.append(dataframe)
        
    nome_arquivo = simpledialog.askstring("Input", "Digite o nome do arquivo:")
    if not nome_arquivo.endswith(".xlsx"):
        nome_arquivo += ".xlsx"
        
    with pd.ExcelWriter(nome_arquivo, engine="openpyxl") as writer:
        for sh in range(len(sheets)):
            files[sh].to_excel(writer, sheet_name=sheets[sh], index=False)
    messagebox.showinfo("", f"Arquivo {nome_arquivo} criado com sucesso!")
    

## Base de Dados

In [1578]:
def base_de_dados():
    messagebox.showinfo("Passo 1", f"Selecione a planilha com as aulas que serão alocadas.")
    df_filename1 = selecionar_arquivo(f"Selecione a planilha com as aulas que serão alocadas.")[1]
    if df_filename1 == "Not gud":
        root.destroy()
        return

    messagebox.showinfo("Passo 2", f"Selecione a planilha com os dados do JúpiterWeb.")
    df_filename2 = selecionar_arquivo(f"Selecione a planilha com as aulas que serão alocadas.")[1]
    if df_filename2 == "Not gud":
        root.destroy()
        return

    # messagebox.showinfo("Passo 3", f"Digite o nome do arquivo da base de dados do modelo.")
    df_filename3 = simpledialog.askstring("Passo 3", f"Digite o nome do arquivo da base de dados do modelo.")
    if not df_filename3:
        messagebox.showwarning("Aviso", "Nenhum nome foi fornecido.")
        root.destroy()
        return
        
    if not df_filename3.endswith(".xlsx"):
        df_filename3 += ".xlsx"
    # df, df_filename = selecionar_arquivo(f"Selecione a planilha com as aulas que serão alocadas.")
    # print([df_filename1, df_filename2, df_filename3])
    try:
        # subprocess(["python", "jupiter sheet maker.py", [df_filename1, df_filename2, df_filename3]])
        subprocess.run(["python", "jupiter sheet maker.py", df_filename1, df_filename2, df_filename3])
        messagebox.showinfo("Sucesso!", f"Base de dados para o modelo criada com sucesso. Verifique o arquivo {df_filename3}.")
    except Exception as e:
        messagebox.showerror("Erro", f"Erro ao executar o script: {e}")

#     if nome_arquivo:  # Se o usuário forneceu o nome
#         nome_arquivo += ".txt"  # Adiciona a extensão .txt
#         try:
#             # Passa o nome do arquivo como argumento para o script 'executar.py'
#             subprocess.run(["python", "executar.py", nome_arquivo])
#             messagebox.showinfo("Sucesso", f"Arquivo '{nome_arquivo}' criado com sucesso!")
#         except Exception as e:
#             messagebox.showerror("Erro", f"Erro ao executar o script: {e}")
#     else:
#         messagebox.showwarning("Aviso", "Nenhum nome foi fornecido.")

## Execução do Modelo

In [1579]:
def executar_modelo():
    messagebox.showinfo("", "Selecione a base de dados para o modelo.")
    base_de_dados, filename = selecionar_arquivo("Selecione a base de dados para o modelo.")
    if filename == "Not gud":
        messagebox.showwarning("Aviso", "Nenhum nome foi fornecido.")
        root.destroy()
        return
    # Criar a janela para exibir a saída
    output_window = tk.Toplevel(root)
    output_window.title("Saída do Script")
    output_text = scrolledtext.ScrolledText(output_window, width=120, height=40)
    output_text.pack()
    # try:
    #     # subprocess(["python", "jupiter sheet maker.py", [df_filename1, df_filename2, df_filename3]])
    #     subprocess.run(["python", "Modelo 1.py", filename])
    #     messagebox.showinfo("Sucesso!", f"Resultados do modelo obtidos!")
    # except Exception as e:
    #     messagebox.showerror("Erro", f"Erro ao executar o script: {e}")
    try:
        # Executar o script como subprocess
        process = subprocess.Popen(
            ["python", "Modelo 1.py", filename],  # Substitua pelo seu script
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        
        # Capturar e exibir a saída linha por linha
        for line in iter(process.stdout.readline, ""):
            if line.strip():
                output_text.insert(tk.END, line)
                output_text.see(tk.END)  # Rolar automaticamente até o final
                root.update()  # Atualizar a interface

        # Exibir erros, se houver
        stderr = process.communicate()[1]
        if stderr:
            output_text.insert(tk.END, "\n[ERRO]:\n" + stderr)
            output_text.see(tk.END)

    except Exception as e:
        output_text.insert(tk.END, f"\nErro ao executar o script: {e}")

## Construção da Análise de Pior Caso

In [1580]:
def pior_caso():
    nova_janela = tk.Toplevel(root)
    nova_janela.title("Selecionar Arquivo e Inserir Valor")
    
    # Frame para o botão e o campo
    frame = tk.Frame(nova_janela)
    frame.pack(pady=10, padx=10)

    # Variável para armazenar o nome do arquivo
    arquivo_selecionado1 = tk.StringVar(value="Selecione a primeira base de dados")
    valor_importante1 = tk.StringVar()
    arquivo_selecionado2 = tk.StringVar(value="Selecione a segunda base de dados")
    valor_importante2 = tk.StringVar()

    # Função para selecionar arquivo
    def selecionar_arquivo1():
        arquivo = filedialog.askopenfilename(title="Selecione a primeira base de dados")
        if arquivo:
            arquivo_selecionado1.set(arquivo)
    def selecionar_arquivo2():
        arquivo = filedialog.askopenfilename(title="Selecione a segunda base de dados")
        if arquivo:
            arquivo_selecionado2.set(arquivo)

    # Botão para selecionar o primeiro arquivo
    btn_selecionar1 = tk.Button(frame, textvariable=arquivo_selecionado1, command=selecionar_arquivo1, wraplength=250, width=40)
    btn_selecionar1.grid(row=0, column=0, padx=5, pady=5)

    # Campo para inserir o valor
    lbl_valor1 = tk.Label(frame, text="Insira o ano da primeira base de dados:")
    lbl_valor1.grid(row=1, column=0, sticky="w", pady=5)

    campo_valor1 = tk.Entry(frame, textvariable=valor_importante1)
    campo_valor1.grid(row=1, column=1, pady=5)

    # Botão para selecionar o segundo arquivo
    btn_selecionar2 = tk.Button(frame, textvariable=arquivo_selecionado2, command=selecionar_arquivo2, wraplength=250, width=40)
    btn_selecionar2.grid(row=2, column=0, padx=5, pady=5)

    # Campo para inserir o valor
    lbl_valor2 = tk.Label(frame, text="Insira o ano da segunda base de dados:")
    lbl_valor2.grid(row=3, column=0, sticky="w", pady=5)

    campo_valor2 = tk.Entry(frame, textvariable=valor_importante2)
    campo_valor2.grid(row=3, column=1, pady=5)

    
    # Função para salvar valores e fechar a janela
    def salvar_valores():
        if not arquivo_selecionado1.get() or arquivo_selecionado1.get() == "Selecione a primeira base de dados":
            messagebox.showwarning("Aviso", "Por favor, selecione a primeira base de dados.")
            return
        if not valor_importante1.get():
            messagebox.showwarning("Aviso", "Por favor, insira o ano da primeira base de dados.")
            return
        if not arquivo_selecionado2.get() or arquivo_selecionado2.get() == "Selecione a segunda base de dados":
            messagebox.showwarning("Aviso", "Por favor, selecione a segunda base de dados.")
            return
        if not valor_importante2.get():
            messagebox.showwarning("Aviso", "Por favor, insira o ano da segunda base de dados.")
            return
        
        # Exemplo de uso das variáveis
        print(f"Primeira base de dados: {arquivo_selecionado1.get()}")
        print(f"Ano da primeira base de dados: {valor_importante1.get()}")
        print(f"Segunda base de dados: {arquivo_selecionado2.get()}")
        print(f"Ano da segunda base de dados: {valor_importante2.get()}")

        sheets = ["SME", "SMA", "SCC", "SSC"]
        df1 = pd.read_excel(arquivo_selecionado1, sheet_name=sheets)
        df1 = pd.concat(df1.values(), ignore_index=True)
        df2 = pd.read_excel(arquivo_selecionado2, sheet_name=sheets)
        df2 = pd.concat(df2.values(), ignore_index=True)
        
        messagebox.showinfo("Sucesso", "Valores salvos com sucesso!")
        nova_janela.destroy()

    # Botão para salvar
    btn_salvar = tk.Button(nova_janela, text="Salvar", command=salvar_valores)
    btn_salvar.pack(pady=10)

    # print(f"ahaha Arquivo selecionado: {arquivo_selecionado.get()}")
    # print(f"ahaha Valor importante: {valor_importante.get()}")

## Botão Teste

In [1581]:
# def teste():
#     # def criar_janela_personalizada():
#     # Nova janela para o botão e campo
#     nova_janela = tk.Toplevel(root)
#     nova_janela.title("Selecionar Arquivo e Inserir Valor")
    
#     # Frame para o botão e o campo
#     frame = tk.Frame(nova_janela)
#     frame.pack(pady=10, padx=10)

#     # Variável para armazenar o nome do arquivo
#     arquivo_selecionado = tk.StringVar(value="Selecione um arquivo")
#     valor_importante = tk.StringVar()

#     # Função para selecionar arquivo
#     def selecionar_arquivo():
#         arquivo = filedialog.askopenfilename(title="Selecione um arquivo")
#         if arquivo:
#             arquivo_selecionado.set(arquivo)

#     # Botão para selecionar o primeiro arquivo
#     btn_selecionar = tk.Button(frame, textvariable=arquivo_selecionado, command=selecionar_arquivo, wraplength=250, width=40)
#     btn_selecionar.grid(row=0, column=0, padx=5, pady=5)

#     # Campo para inserir o valor
#     lbl_valor = tk.Label(frame, text="Insira um valor:")
#     lbl_valor.grid(row=1, column=0, sticky="w", pady=5)

#     campo_valor = tk.Entry(frame, textvariable=valor_importante)
#     campo_valor.grid(row=1, column=1, pady=5)

    
#     # Função para salvar valores e fechar a janela
#     def salvar_valores():
#         if not arquivo_selecionado.get() or arquivo_selecionado.get() == "Selecione um arquivo":
#             messagebox.showwarning("Aviso", "Por favor, selecione um arquivo.")
#             return
#         if not valor_importante.get():
#             messagebox.showwarning("Aviso", "Por favor, insira um valor.")
#             return
        
#         # Exemplo de uso das variáveis
#         print(f"Arquivo selecionado: {arquivo_selecionado.get()}")
#         print(f"Valor importante: {valor_importante.get()}")
        
#         messagebox.showinfo("Sucesso", "Valores salvos com sucesso!")
#         nova_janela.destroy()

#     # Botão para salvar
#     btn_salvar = tk.Button(nova_janela, text="Salvar", command=salvar_valores)
#     btn_salvar.pack(pady=10)

#     # root2.mainloop()
#     # # Checkbox 1
#     # checkbox1 = Checkbutton(root, text="Opção 1", variable=var1, command=atualizar_estado)
#     # checkbox1.grid(row=0, column=0, sticky="w")
#     # input1 = Entry(root, state=DISABLED)  # Input associado ao Checkbox 1
#     # input1.grid(row=0, column=1)

## Filtra Dataframe

In [1582]:
def filtra_dataframe(df):
    # Talvez seja necessário uma filtragem melhor, por conta das aulas ministradas na Área 2.
    # curriculos = ['BMACC', 'BMA', 'LMA', 'MAT-NG', 'BECD', 'BCC', 'BSI', 'BCDados']
    # Criar uma expressão regular com as palavras-chave
    # filtro = '|'.join(curriculos)
    
    # Filtrar as linhas onde a coluna contém qualquer palavra-chave.
    # df_filtrado = df[df['Curso(s)'].str.contains(filtro, case=False, na=False)]

    df_filtrado = df[df['Deve ser alocada no ICMC?'] == 'X']

    return df_filtrado

### Primeira Criar Visualização Vazia

In [1583]:
# def criar_visualizacao_vazia(file_path):
#     # Os próximos códigos são feitos para criar arquivos de planilha bem semelhantes,
#     # então fiz uma pequena coletânea de funções que são recorrentemente usadas.
    
#     # Leio novamente a planilha contendo a solução do modelo.
#     # dfv = pd.read_excel('C:/Users/gabri/Estágio/Códigos/dados_solucao - m1.2.xlsx')
#     salas = pd.read_excel('C:/Users/gabri/Estágio/Códigos/Endgame/base apos consolidacao.xlsx', sheet_name="Salas")
#     dfv = pd.read_excel(file_path)
    
#     # Ordena o DataFrame com base na coluna desejada, no caso, 'Sala'.
#     dfv = dfv.sort_values(by='Sala')
    
#     # Cria uma lista dos valores únicos da coluna ordenada, isto é, uma lista das salas utilizadas na solução.
#     salas1 = dfv['Sala'].unique().tolist()
    
#     # Configurar horário inicial e final, isto é, o intervalo dos horários que serão colocados na planilha para a visualização.
#     start_time = datetime.strptime("07:00", "%H:%M")
#     end_time = datetime.strptime("23:30", "%H:%M")
    
#     # Crio uma lista para conter os valores do intervalo de horários.
#     horarios = []
#     # Crio uma variável com o horário inicial.
#     current_time = start_time
#     # Enquanto o horário inicial não for maior que o horário final, isto é, enquanto houver valores para serem colocados no intervalo.
#     while current_time <= end_time:
#         # Adiciono o horário atual na lista de horários.
#         horarios.append(current_time.strftime("%H:%M"))
#         # Faço um acréscimo de 30 minutos no horário atual.
#         current_time += timedelta(minutes=30)
    
#     # print(horarios)
    
#     # Função para padronizar o horário
#     def padronizar_horario_intranet(horario):
#         # Para padronizar e facilitar a leitura dos horários, retiro espaços do horário.
#         if ' ' in horario:
#             horario = str(horario).replace(' ', '')
#         # Para padronizar e facilitar a leitura dos horários, substituo 'h' por ':', caso tenha no horário.
#         if 'h' in horario:
#             horario = str(horario).replace('h', ':')
#         # Separo o dia do horário da aula.
#         dia, intervalo = horario.split('-')
#         # Separo o começo e o fim da aula.
#         start, end = intervalo.split('/')
#         # Traduzo o horário para um objeto de data, ou seja, crio um objeto com propriedades e métodos dedicados à manipulação de horários do dia.
#         start_dt = datetime.strptime(start, "%H:%M")
#         end_dt = datetime.strptime(end, "%H:%M")
#         # horario_dt = datetime.strptime(h, "%H:%M")
    
#         # Começo uma análise para ver como o horário fica mais adequado.
#         # Se o horário de início da disciplina é menor que 18, ou seja, 8:10,10:10,14:20 e 16:20, coloco os minutos em 0.
#         if start_dt.hour < 18:
#             start_dt = start_dt.replace(minute=0)
#         # Caso contrário, isto é, 18, 19 e 21, mantém normal.
    
#         # Se o horário de fim da aula é menor ou igual que 18, ou seja, 9:50, 11:50, 16 e 18
#         if end_dt.hour <= 18:
#             # Se os minutos do horário de fim da aula são inferiores a 30.
#             if end_dt.minute < 30:
#                 # Arredondo o horário para a hora anterior, com 30 minutos.
#                 # Ex: 18:00 se torna 17:30.
#                 end_dt = end_dt.replace(minute=30,hour=end_dt.hour - 1)
#             # Se os minutos do horário de fim são superiores a 30.
#             elif end_dt.minute > 30:
#                 # Apenas deixo os minutos do horário como 30.
#                 # Ex: 09:50 se torna 09:30
#                 end_dt = end_dt.replace(minute=30)
#         # Se o horário de fim da aula é maior que 18, ou seja, 19, 20:40 e 22:40
#         else:
#             # Se os minutos do horário de fim são inferiores a 30, mas superiores a 0.
#             if end_dt.minute < 30 and end_dt.minute > 0:
#                 # Apenas deixo os minutos do horário como 0.
#                 # Ex: 19:10 se torna 19:00
#                 end_dt = end_dt.replace(minute=0)
#             # Se os minutos do horário de fim são superiores a 30.
#             elif end_dt.minute > 30:
#                 # Apenas deixo os minutos do horário como 30.
#                 # Ex: 20:40 se torna 20:30
#                 end_dt = end_dt.replace(minute=30)
#             # Se os minutos forem exatamente 0, eu considero que a aula acaba meia hora antes do denotado.
#             # Ex: fim às 19:00 se torna 18:30 para evitar problemas com disciplinas que começam às 19:00
#             else:
#                 end_dt = end_dt.replace(minute=30,hour=end_dt.hour - 1)
#         # start_dt.replace(hour=start_dt.hour, minute=start_dt.minute).s
#         # end_dt.replace(hour=end_dt.hour, minute=end_dt.minute)
#         # return str(f'{dia} - {horario_ajustado.strftime("%H:%M")}')
#         # Retorno o novo horário traduzido.
#         return str(f'{dia} - {start_dt.strftime("%H:%M")}/{end_dt.strftime("%H:%M")}')
    
#     # Crio uma variável com a quantidade de colunas necessárias para colocar todos os horários do intervalo.
#     sala_colunas = len(horarios)
#     # Lista com os nomes dos dias da semana que serão usados na visualização.
#     dias_semana = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta", "Sábado"]
    
#     # Função para preencher a planilha com os nomes das disciplinas.
#     def preencher_planilha_vazia(start_row, start_column, end_row, end_column, start, sala, dias_semana, horarios, aula, disciplina, ws, aulas, df_vazio):
#         # Defino um preenchimento verde para preencher as células com o nome das disciplinas.
#         # green_fill = PatternFill(start_color="99CC00", end_color="99CC00", fill_type="solid")
#         blank_fill = PatternFill(start_color="00ffffff", end_color="00ffffff", fill_type="solid")
#         yellow_fill = PatternFill(start_color="00E3DE00", end_color="00E3DE00", fill_type="solid")
    
#         # Defino um estilo de borda para colocar nas células.
#         thin_border = Border(
#             left=Side(style="thin"),
#             right=Side(style="thin"),
#             top=Side(style="thin"),
#             bottom=Side(style="thin")
#         )
    
#     #     sala = "Sala 101"
#         # Mesclo as células na posição correta para colocar o número/nome da sala.
#         ws.merge_cells(start_row=start_row, start_column=start_column, end_row=end_row, end_column=end_column)
#         # Adiciono o número/nome da sala na célula mesclada.
#         ws.cell(row=start_row, column=start_column).value = sala
#         # Defino o alinhamento do texto da célula mesclada para deixar o número/nome da sala no centro da célula.
#         ws.cell(row=start_row, column=start_column).alignment = Alignment(horizontal="center", vertical="center")
#         # Por conta da célula mesclada funcionar de forma diferente de uma célula normal, é necessário aplicar o estilo da borda
#         # em todas as células que foram mescladas.
#         for col in range(1, end_column):
#             ws.cell(row=start_row, column=col+1).border = thin_border
    
#         # Preencho a primeira coluna com dias da semana.
#         for i, dia in enumerate(dias_semana, start=start):
#             ws.cell(row=i, column=start_column-1).value = dia
#             ws.cell(row=i, column=start_column-1).border = thin_border
    
#         # Preencho o horário nas colunas das células que foram mescladas, semelhante a um cabeçalho.
#         for j, horario in enumerate(horarios, start=start_column):
#             ws.cell(row=start_row+1, column=j).value = horario
#             ws.cell(row=start_row+1, column=j).alignment = Alignment(horizontal="center", vertical="center")
#             ws.cell(row=start_row+1, column=j).border = thin_border
    
#         # Para todas as linhas da tabela.
#         for row in range(start, start + len(dias_semana)):
#             # Para todas as colunas da tabela.
#             for col in range(start_column, start_column + len(horarios)):
#                 # Aplico a borda.
#                 ws.cell(row=row, column=col).border = thin_border
#                 # ws.cell(row=row, column=col).fill = yellow_fill
#                 ws.cell(row=row, column=col).fill = blank_fill
#         # print(horarios)
    
#         # Preenchimento das aulas nas planilhas.
#         # Para cada horário de aula na lista de aulas da sala
#         for h in aula:
#             # print(h)
#             # Separo o dia e o intervalo da aula.
#             dia_h, int_h = h.split(' - ')
#             # Separo o horário de fim e de início da aula.
#             start_h, end_h = int_h.split('/')
#             # print(type(start_h), end_h)
#             # Para cada dia da semana.
#             for i, dia in enumerate(dias_semana, start=start):
#                 # Verifico se o dia em questão é o mesmo que o da aula sendo analisada.
#                 if ws.cell(row=i, column=start_column-1).value == dia_h:
#                     # Se for, salvo o índice das colunas de início e fim do intervalo da aula, isto é,
#                     # identifico os índices das células que serão mescladas para colocar o nome da turma/disciplina da aula.
#                     # Ex: uma aula de Terça das 08:10 às 09:50 é considerada como das 08:00 às 09:30,
#                     # então as células mescladas são as da linha de Terça, começando na coluna das 08:00 até a coluna das 09:30.
#                     index_start = horarios.index(start_h)
#                     index_end = horarios.index(end_h)
    
#                     # Mesclo as células necessárias.
#                     ws.merge_cells(start_row=i,start_column=index_start+2, end_row=i, end_column=index_end+2)
#                     # Defino uma variável para simbolizar a célula do topo esquerdo da célula mesclada (que é a que deve ser alterada, neste caso).
#                     merged_cell = ws.cell(row=i,column=index_start+2)
#                     # Escrevo o nome da turma/disciplina da aula em questão, aplico o preenchimento verde, e o alinhamento de centro.
#                     # merged_cell.value = disciplina[aula.index(h)]
#                     # merged_cell.fill = blank_fill
#                     merged_cell.fill = yellow_fill
#                     # merged_cell.alignment = Alignment(horizontal="center", vertical="center")
    
#                     # Contabilizo a aula colocada na planilha.
#                     aulas += 1
#                 else:
#                     # ws.cell(row=i, column=start_column).fill = yellow_fill
#                     ws.cell(row=i, column=start_column).fill = blank_fill

#         # Função para obter a cor da célula
#         def get_cell_color(cell):
#             fill = cell.fill
#             # if fill and fill.start_color.index != "ffffff":  # Ignorar células sem cor
#             if fill and fill.start_color.index != "00E3DE00":  # Ignorar células amarelas
#                 return fill.start_color.index  # Retorna o código da cor
#             return None  # Sem cor definida
        
#         # Percorrer as linhas da planilha
#         for row in ws.iter_rows(min_row=start, max_row=start+5):
#         # for i, dia in enumerate(dias_semana, start=start):
#             start_col = None  # Início da mesclagem
#             # current_color = None  # Cor atual
        
#             for col_idx, cell in enumerate(row, start=1):
#                 cell_color = get_cell_color(cell)
#                 # print(f"A cor dessa célula é essa: {cell_color}")
#                 if cell_color == "00ffffff":  # Se for a mesma cor, continua
#                     if start_col is None:
#                         start_col = col_idx
#                     # continue
#                 else:
#                     if start_col is not None and col_idx - start_col > 1:
#                         # ws.merge_cells(start_row=cell.row, start_column=start_col, 
#                         #                end_row=cell.row, end_column=col_idx - 1)
#                         # Aqui é onde eu vou pegar os intervalos do merge
#                         df_vazio.loc[len(df_vazio)] = [f"{sala}", f"{ws.cell(row=cell.row, column=1).value}", f"{horarios[start_col-2]} - {horarios[col_idx-1-2+1]}"]
#                         # int_disponivel = f"Sala {sala}, horário disponível {ws.cell(row=cell.row, column=1).value}"
#                         # int_disponivel += f" - {horarios[start_col-2]}/{horarios[col_idx-1-2]}"
#                         # print(int_disponivel)
#                     start_col = None  # Atualiza o início da mesclagem
#                     # current_color = cell_color  # Atualiza a cor atual
        
#             # Mesclar o último grupo da linha, se necessário
#             if start_col is not None and start_col < len(row):
#                 # ws.merge_cells(start_row=cell.row, start_column=start_col, 
#                 #                end_row=cell.row, end_column=len(row))
#                 df_vazio.loc[len(df_vazio)] = [f"{sala}", f"{ws.cell(row=cell.row, column=1).value}", f"{horarios[start_col-2]} - {horarios[col_idx-1-2+1]}"]
#                 # int_disponivel = f"Sala {sala}, horário disponível {ws.cell(row=cell.row, column=1).value}"
#                 # int_disponivel += f" - {horarios[start_col-2]}/{horarios[col_idx-1-2+1]}"
#                 # print(int_disponivel)
#         # Retorno o número de aulas colocadas na planilha dessa forma.
#         return aulas
    
#     # Função que define a largura das colunas da planilha.
#     def ajusta_largura(ws):
#         # Para cada coluna na planilha.
#         for col in ws.columns:
#             # Defino uma variável de comprimento máximo
#             max_length = 0
#             # Verifico se a primeira célula da coluna é uma célula mesclada.
#             if type(col[0]) == openpyxl.cell.cell.MergedCell:
#                 # Se for, obtenho a letra da coluna pela célula abaixo, pois uma célula mesclada não possui o método necessário para obtê-la.
#                 column = col[1].column_letter  # Obtém a letra da coluna
#             else:
#                 # Caso contrário, obtenho a letra da coluna pela primeira célula mesmo.
#                 column = col[0].column_letter
    
#             # Para cada célula da coluna.
#             for cell in col:
#                 # Utilizo o método try-catch para ignorar erros ao analisar uma célula mesclada, já que ela funciona de forma diferente.
#                 try:
#                     # Verifico se o comprimento do conteúdo da célula é maior que meu comprimento máximo.
#                     if len(str(cell.value)) > max_length:
#                         # Em caso positivo, atualizo o comprimento máximo.
#                         max_length = len(str(cell.value))
#                 except:
#                     pass
#             # Com o comprimento máximo, atualizo a largura das colunas da planilha para serem de um tamanho próximo ao mais alto.
#             ws.column_dimensions[column].width = (max_length) * 1.1
    
#     # def merge_yellow_cells(ws):
#     #     # Função para obter a cor da célula
#     #     def get_cell_color(cell):
#     #         fill = cell.fill
#     #         if fill and fill.start_color.index != "ffffff":  # Ignorar células sem cor
#     #             return fill.start_color.index  # Retorna o código da cor
#     #         return None  # Sem cor definida
        
#     #     # Percorrer as linhas da planilha
#     #     for row in ws.iter_rows(min_row=1, max_row=ws.max_row):
#     #         start_col = None  # Início da mesclagem
#     #         current_color = None  # Cor atual
        
#     #         for col_idx, cell in enumerate(row, start=1):
#     #             cell_color = get_cell_color(cell)
#     #             # print(f"A cor dessa célula é essa: {cell_color}")
#     #             if cell_color == "00E3DE00":  # Se for a mesma cor, continua
#     #                 if start_col is None:
#     #                     start_col = col_idx
#     #                 # continue
#     #             else:
#     #                 if start_col is not None and col_idx - start_col > 1:
#     #                     ws.merge_cells(start_row=cell.row, start_column=start_col, 
#     #                                    end_row=cell.row, end_column=col_idx - 1)
#     #                 start_col = None  # Atualiza o início da mesclagem
#     #                 # current_color = cell_color  # Atualiza a cor atual
        
#     #         # Mesclar o último grupo da linha, se necessário
#     #         if start_col is not None and start_col < len(row):
#     #             ws.merge_cells(start_row=cell.row, start_column=start_col, 
#     #                            end_row=cell.row, end_column=len(row))
#     # Cria o workbook e a sheet, isto é, um objeto de planilha do Excel com a planilha ativa.
#     wb = Workbook()
#     ws = wb.active
#     ws.title = "Horário de Aulas"
    
#     # Defino as linhas e colunas de início e fim para a primeira tabela da planilha, isto é, para preencher os dados da primeira sala.
#     start_row = 1 # Linha de início.
#     start_column = 2 # Coluna de início.
#     end_row = 1 # Linha de término.
#     end_column = 1 + sala_colunas # Coluna de término.
#     start = start_row + 2 # Linha de início para listar os dias da semana.
#     space_between = 3 # Número de linhas entre a tabela de uma sala para a de outra sala.
    
#     # A variável aulas é um contador de quantas aulas foram colocadas na planilha, facilitando a verificação.
#     aulas = 0
#     df_vazio = pd.DataFrame(columns=["Sala", "Dia da semana", "Horário vago"])
    
#     # Para cada sala utilizada na solução.
#     for sala in salas1:
#         # A variável df_filtrado é um dataframe com as linhas de dados do dataframe da solução, cuja coluna é a mesma que a sala.
#         # Em outras palavras, é um dataframe com as aulas que foram alocadas na sala atual.
#         df_filtrado = dfv[dfv['Sala'] == sala]
    
#         # A variável aula é uma lista dos horários das aulas após serem padronizados.
#         aula = df_filtrado['Horário'].apply(lambda x: padronizar_horario_intranet(x)).tolist()
    
#         # A variável disciplina é uma lista do código das disciplinas cujas aulas foram alocadas na sala atual.
#         disciplina = df_filtrado['Disciplina'].tolist()
    
#         # Chamo a função que preenche a planilha, mandando os parâmetros necessários, e retornando o número de aulas alocadas na planilha.
#         aulas = preencher_planilha_vazia(start_row, start_column, end_row, end_column, start, sala, dias_semana, horarios, aula, disciplina, ws, aulas, df_vazio)
    
#         # Feito isso, ttualizo meus dados de criação, isto é, as coordenadas de onde a tabela da sala seguinte será colocada na planilha.
#         start_row = start_row + 2 + len(dias_semana) + space_between
#         end_row = start_row
#         start = start_row + 2
    
#     for sala in salas['Sala'].tolist():
#         if sala not in salas1:
#             if sala == '6-303' and '6-303/6-304' in salas1:
#                 continue
#             elif sala == '6-304' and '6-303/6-304' in salas1:
#                 continue
#             elif sala == '6-305' and '6-305/6-306' in salas1:
#                 continue
#             elif sala == '6-306' and '6-305/6-306' in salas1:
#                 continue
#             else:
#                 df_vazio.loc[len(df_vazio)+1, 'Sala'] = f"A sala {sala} não foi usada pela graduação."

    
#     # Depois de todas as aulas terem sido alocadas, ajusto a largura das colunas da planilha.
#     ajusta_largura(ws)
#     # merge_yellow_cells(ws)
#     # Por fim, salva o arquivo.
#     # Por fim, salva o arquivo.
#     # sala_coord = []
#     # start_row = 1
#     # for _ in range(len(salas1)):
#     #     sala_coord.append(ws.cell(row=start_row, column=2).value)
#     #     start_row = start_row + 2 + len(dias_semana) + space_between

#     # print(sala_coord)
#     caminho_arquivo = f"Visualização de Espaços Livres - Modelo 4"
#     caminho_arquivo1 = f"Planilha com Espaços Livres - Modelo 4"
    
#     full_name = caminho_arquivo + ".xlsx"
#     i = 1
#     while os.path.exists(full_name):
#         full_name = f"{caminho_arquivo} ({i}){'.xlsx'}"
#         i += 1
#     caminho_arquivo = full_name

#     full_name = caminho_arquivo1 + ".xlsx"
#     i = 1
#     while os.path.exists(full_name):
#         full_name = f"{caminho_arquivo1} ({i}){'.xlsx'}"
#         i += 1

#     caminho_arquivo1 = full_name
    
#     wb.save(caminho_arquivo)
#     df_vazio.to_excel(caminho_arquivo1, sheet_name="Resultados", index=False)
#     # pd.to_excel
#     print(f"Arquivo '{caminho_arquivo}' salvo com sucesso!")
#     messagebox.showinfo("", f"Arquivo {caminho_arquivo} criado com sucesso!\n\nArquivo {caminho_arquivo1} criado com sucesso!")
#     # print("Número de aulas alocadas:", aulas)

# Teste de Interface mais Clara

## Padroniza dataframe

In [1584]:
# Função que padroniza os dataframes para terem o mesmo formato.
def padroniza_dataframe(file_name, header_row, ano):
    # Leio o dataframe correto, com a linha do cabeçalho.
    df = pd.read_excel(file_name, header=header_row)

    # Primeiramente, filtro o dataframe para fazer a edição apenas nas disciplinas que importam.
    # Isto é, eu passo a trabalhar apenas com as disciplinas marcadas com um 'X' que devem ser alocadas no ICMC.
    df = df[df['Deve ser alocada no ICMC?'] == 'X']
    
    # Para cada cabeçalho/coluna 'col' do dataframe.
    for col in range(len(df.columns)):
        # Verifico se tem um "\n" no texto do dataframe.
        if "\n" in df.columns[col]:
            # Se houver, substituo ele por um " ".
            df = df.rename(columns={df.columns[col] : df.columns[col].replace("\n", " ")})
        # Verifico se estou na coluna das salas - "Sala (a definir)".
        if "Sala" in df.columns[col]:
            # Se estiver, substituo o nome da coluna por apenas "Sala".
            df = df.rename(columns={df.columns[col] : "Sala"})
            
    # Após editar os cabeçalhos, verifico se existe um cabeçalho chamado "Horário 4" no dataframe.
    if "Horário 4" not in df.columns:
        # Se não houver, eu o adiciono no dataframe. Primeiro, procuro pelo cabeçalho "Horário 3", e insiro a nova coluna ao lado.
        df.insert(df.columns.get_loc("Horário 3") + 1, "Horário 4", "")

    # Salvo o nome dos cabeçalhos do dataframe após as edições.
    headers = df.columns
    
    # Preencho os espaços vazios da coluna de uso de laboratórios com "Não". Dessa forma, o usuário só precisa indicar qual disciplina
    # requer um laboratório.
    df['Utilizará laboratório? (sim ou não)'] = df['Utilizará laboratório? (sim ou não)'].fillna("Não")
    # print(df['Utilizará laboratório? (sim ou não)'])
    
    # Busco e listo por disciplinas cuja Turma não foi definida.
    turmas0 = df[df['Turma'].isna()].index.tolist()
    # Se existirem tais disciplinas, uma janela é aberta para indicar as linhas do arquivo que possuem turmas vazias.
    if len(turmas0) != 0:
        messagebox.showwarning(f"Aviso!", f"A(s) linha(s) {[d + header_row + 2 for d in turmas0]} do arquivo {file_name} \
        possuem turmas não identificadas. Verifique qual a turma da disciplina.")
        return None

    # Adiciono mais duas colunas no dataframe, uma para colocar o número de inscritos das disciplinas, e outra para o ano dos dados.
    df.insert(df.columns.get_loc(headers[-1]), "Vagas por disciplina", "")
    df.insert(df.columns.get_loc(headers[-1]), "Ano dos dados", ano)

    # Alguns horários das disciplinas podem ter sido definidos com um traço diferente do usual do teclado.
    # Por isso, eu busco em cada coluna de horários por esses possíveis traços errados, pois eles prejudicam a leitura das disciplinas.
    # Para cada coluna de horário:
    for header in ['Horário 1', 'Horário 2', 'Horário 3', 'Horário 4']:
        # Para cada linha do dataframe:
        for d in df.index:
            # Verifico se o traço anormal está na célula em questão.
            if "–" in str(df.loc[d, header]):
                # Se estiver, substituo-o pelo traço normal.
                df.loc[d, header] = df.loc[d, header].replace('–', '-')

    # Adiciono 4 colunas para anotar possíveis salas onde as aulas ficam PROIBIDAS de serem alocadas.
    # Cada coluna refere-se a uma coluna de horários, fazendo com que as salas listadas em uma coluna afetem apenas a aula de mesma coluna.
    # Ex: Adicionar a sala 3-009 na segunda coluna de proibição faz com que a aula do Horário 2 de uma disciplina não possa ser na 3-009.
    df.insert(df.columns.get_loc(headers[-1]), "Proibir Horário 1", pd.NA)
    df.insert(df.columns.get_loc(headers[-1]), "Proibir Horário 2", pd.NA)
    df.insert(df.columns.get_loc(headers[-1]), "Proibir Horário 3", pd.NA)
    df.insert(df.columns.get_loc(headers[-1]), "Proibir Horário 4", pd.NA)
    
    # Retorno o dataframe com todas as padronizações necessárias.
    return df

## Tooltip

In [1585]:
# Classe Tooltip utilizada para mostrar textos ao sobrevoar o mouse em alguma parte da interface.
class Tooltip:
    def __init__(self, widget, text):
        self.widget = widget
        self.text = text
        self.tooltip_window = None

        # Eventos para mostrar e ocultar o tooltip
        widget.bind("<Enter>", self.show_tooltip)
        widget.bind("<Leave>", self.hide_tooltip)

    def show_tooltip(self, event=None):
        # Cria a janela do tooltip
        if self.tooltip_window is not None:
            return
        
        x = self.widget.winfo_rootx() + 20  # Posição X do tooltip
        y = self.widget.winfo_rooty() + 20  # Posição Y do tooltip

        self.tooltip_window = tk.Toplevel(self.widget)
        self.tooltip_window.wm_overrideredirect(True)  # Remove bordas da janela
        self.tooltip_window.geometry(f"+{x}+{y}")

        label = tk.Label(
            self.tooltip_window, 
            text=self.text, 
            background="lightyellow", 
            relief="solid", 
            borderwidth=1, 
            font=("Arial", 10)
        )
        label.pack(ipadx=5, ipady=3)

    def hide_tooltip(self, event=None):
        # Destrói a janela do tooltip
        if self.tooltip_window:
            self.tooltip_window.destroy()
            self.tooltip_window = None


## Ler planilhas dos institutos (Base de dados das aulas e do jupiter)

In [1586]:
# Função para criar a base de dados das aulas e a dos dados do JúpiterWeb.
# Seu parâmetro 'jupiter' refere-se a uma variável booleana, que recebe o valor True caso seja o momento de criar a Base de Dados do Júpiter,
# e False caso contrário.
def planilha_dep(jupiter):
    
    # Crio uma nova janela em cima da janela principal da interface.
    nova_janela = tk.Toplevel(root)

    # Baseado em qual base de dados será criada, defino um nome diferente para a janela.
    if not jupiter:
        nova_janela.title("Construir Base de Dados das Aulas")

    else:
        nova_janela.title("Construir Base de Dados do JúpiterWeb")
    
    # Crio o frame para armazenar os botões e outros campos da nova janela.
    frame = tk.Frame(nova_janela)
    frame.pack(pady=10, padx=10)

    # Defino várias variáveis para armazenar os nomes dos arquivos para criar a base de dados.
    arquivo_sme = tk.StringVar(value="Selecione a planilha do SME")
    arquivo_sma = tk.StringVar(value="Selecione a planilha do SMA")
    arquivo_scc = tk.StringVar(value="Selecione a planilha do SCC")
    arquivo_ssc = tk.StringVar(value="Selecione a planilha do SSC")
    arquivo_outros = tk.StringVar(value="Selecione a planilha dos Outros Institutos")
    arquivo_salas = tk.StringVar(value="Selecione a planilha dos dados das salas")
    # Defino uma variável para conter o ano dos dados fornecidos.
    ano_dados = IntVar()
    # Defino uma variável para conter o nome da nova base de dados.
    nome_arquivo = tk.StringVar()
    # Defino uma lista que irá conter os nomes de outros arquivos necessários para criar uma base de dados.
    lista_outros = []

    # Defino as funções para selecionar arquivos.
    def selecionar_sme():
        # A classe filedialog faz com que o gerenciador de arquivos seja aberto para selecionar um arquivo.
        # O usuário seleciona o arquivo contendo a base de dados das aulas.
        arquivo = filedialog.askopenfilename(title="Selecione a planilha do SME")

        # Se um arquivo foi selecionado:
        if arquivo:
            # Salvo o caminho do arquivo.
            arquivo_sme.set(arquivo)
    def selecionar_sma():
        arquivo = filedialog.askopenfilename(title="Selecione a planilha do SMA")
        if arquivo:
            arquivo_sma.set(arquivo)
    def selecionar_scc():
        arquivo = filedialog.askopenfilename(title="Selecione a planilha do SCC")
        if arquivo:
            arquivo_scc.set(arquivo)
    def selecionar_ssc():
        arquivo = filedialog.askopenfilename(title="Selecione a planilha do SSC")
        if arquivo:
            arquivo_ssc.set(arquivo)
    # Se não estiver criando a base de dados do Júpiter:
    if not jupiter:
        # Também defino as funções necessárias para salvar outros arquivos.
        def selecionar_outros():
            arquivo = filedialog.askopenfilename(title="Selecione a planilha dos Outros Institutos")
            if arquivo:
                arquivo_outros.set(arquivo)
        def selecionar_salas():
            arquivo = filedialog.askopenfilename(title="Selecione a planilha dos dados das salas")
            if arquivo:
                arquivo_salas.set(arquivo)
        
    
    
    # Crio uma legenda para ficar ao lado do botão.
    lbl_sme = tk.Label(frame, text="Selecione a planilha do SME")
    # Defino a posição do texto na janela.
    lbl_sme.grid(row=0, column=0, pady=5, sticky="w")
    # Crio o botão para salvar o arquivo do SME.
    btn_selecionar_sme = tk.Button(frame, textvariable=arquivo_sme, command=selecionar_sme, wraplength=250, width=40)
    # Defino a posição do botão na janela.
    btn_selecionar_sme.grid(row=0, column=1, padx=5, pady=5)

    # As linhas a seguir são análogas.
    lbl_sma = tk.Label(frame, text="Selecione a planilha do SMA")
    lbl_sma.grid(row=1, column=0, pady=5, sticky="w")
    btn_selecionar_sma = tk.Button(frame, textvariable=arquivo_sma, command=selecionar_sma, wraplength=250, width=40)
    btn_selecionar_sma.grid(row=1, column=1, padx=5, pady=5)

    lbl_scc = tk.Label(frame, text="Selecione a planilha do SCC")
    lbl_scc.grid(row=2, column=0, pady=5, sticky="w")
    btn_selecionar_scc = tk.Button(frame, textvariable=arquivo_scc, command=selecionar_scc, wraplength=250, width=40)
    btn_selecionar_scc.grid(row=2, column=1, padx=5, pady=5)

    lbl_ssc = tk.Label(frame, text="Selecione a planilha do SSC")
    lbl_ssc.grid(row=3, column=0, pady=5, sticky="w")
    btn_selecionar_ssc = tk.Button(frame, textvariable=arquivo_ssc, command=selecionar_ssc, wraplength=250, width=40)
    btn_selecionar_ssc.grid(row=3, column=1, padx=5, pady=5)
        
    # Como anteriormente, caso a base sendo criada não é a do JúpiterWeb:
    if not jupiter:
        # Defino algumas legendas e botões adicionais.
        lbl_outros = tk.Label(frame, text="Selecione a planilha dos Outros Institutos")
        lbl_outros.grid(row=4, column=0, pady=5, sticky="w")
        btn_selecionar_outros = tk.Button(frame, textvariable=arquivo_outros, command=selecionar_outros, wraplength=250, width=40)
        btn_selecionar_outros.grid(row=4, column=1, padx=5, pady=5)
        
        lbl_salas = tk.Label(frame, text="Selecione a planilha dos dados das salas")
        lbl_salas.grid(row=5, column=0, pady=5, sticky="w")
        btn_selecionar_salas = tk.Button(frame, textvariable=arquivo_salas, command=selecionar_salas, wraplength=250, width=40)
        btn_selecionar_salas.grid(row=5, column=1, padx=5, pady=5)

    
        # Campo para inserir o valor
        lbl_ano = tk.Label(frame, text="Insira o ano dos dados:")
        lbl_ano.grid(row=6, column=0, sticky="w", pady=5)
        campo_ano = tk.Entry(frame, textvariable=ano_dados)
        campo_ano.grid(row=6, column=1, pady=5)

    # Caso esteja sendo criada a base de dados do Júpiter:
    else:
        # Defino um frame adicional na janela.
        frame2 = tk.Frame(nova_janela)
        frame2.pack(pady=10, padx=10)
    
        # Defino algumas funções especiais para utilizar nesse novo frame.
        
        # Defino uma função para salvar um arquivo, da mesma forma como as funções anteriores.
        def add_file():
            file_path = filedialog.askopenfilename(title="Selecione um arquivo")
            # A diferença, é que se um arquivo foi selecionado, eu o salvo em uma lista, ao invés de uma variável única.
            if file_path:
                lista_outros.append(file_path)
                # E também atualizo a lista visualizada no frame utilizando uma outra função definida aqui.
                update_listbox()

        # Defino uma função para remover um arquivo da lista selecionado pelo usuário.
        def remove_selected():
            # Tento executar as linhas a seguir, que só devem ser executadas se o usuário escolher um arquivo da lista.
            try:
                # Salvo o índice do arquivo selecionado pelo usuário.
                selected_index = file_listbox.curselection()[0]
                # Removo o arquivo de mesmo índice da lista.
                lista_outros.pop(selected_index)
                # E atualizo a lista visualizada no frame.
                update_listbox()
            # Se nenhum arquivo ser selecionado, a interação é ignorada, e nada acontece.
            except IndexError:
                pass

        # Defino uma função que atualiza a visualização da lista no frame.
        def update_listbox():
            # Primeiro, limpo a lista que estava sendo mostrada anteriormente.
            file_listbox.delete(0, tk.END)
            # Depois disso, adiciono os arquivos da lista no visor.
            for file in lista_outros:
                file_listbox.insert(tk.END, file)  # Adiciona os arquivos novamente
        # def concluir():
        #     # lista_outros = list(file_listbox).copy()
        #     for l in lista_outros:
        #         arquivo_outros += l + '\n'
        #     print(lista_outros)
            
        # Defino um botão para adicionar arquivos, e sua posição na janela.
        add_file_button = tk.Button(frame2, text="Adicionar Arquivo", command=add_file)
        add_file_button.grid(row=0, column=0, pady=5, sticky="w")
    
        # Defino uma lista para aparecer no visor do frame.
        file_listbox = tk.Listbox(frame2, width=100, height=10)
        file_listbox.grid(row=1, column=0, pady=5)
        
        # Defino um botão para remover arquivos, e sua posição na janela.
        remove_file_button = tk.Button(frame2, text="Remover Selecionado", command=remove_selected)
        remove_file_button.grid(row=0, column=0, pady=5, sticky="e")
    
        # # Botão para concluir update
        # update_file_button = tk.Button(frame2, text="Concluir", command=concluir)
        # update_file_button.grid(row=2, column=0, pady=5, sticky="ew")
    #     lbl_outros = tk.Label(frame, text="Selecione a planilha dos Outros Institutos")
    #     lbl_outros.grid(row=4, column=0, pady=5)
    #     btn_selecionar_outros = tk.Button(frame, textvariable=arquivo_outros, 
    #                                       command=selecionar_outros_jpter(lista_outros=lista_outros, 
    #                                                                 arquivo_outros=arquivo_outros, 
    #                                                                 nova_janela=nova_janela)
    #                                       , wraplength=250, width=40)
    #     btn_selecionar_outros.grid(row=4, column=1, padx=5, pady=5)
        
    # Defino uma legenda e sua posição na janela, que indicará o campo onde o usuário digitará o nome do arquivo da base de dados.
    lbl_arq = tk.Label(frame, text="Insira o nome para a base de dados:")
    lbl_arq.grid(row=7, column=0, sticky="w", pady=5)
    campo_arq = tk.Entry(frame, textvariable=nome_arquivo)
    campo_arq.grid(row=7, column=1, pady=5)

    
    # Defino uma função que salva os valores das variáveis contendo o nome dos arquivos escolhidos.
    def salvar_valores():
        # Todas as condições a seguir seguem a lógica de que, se um arquivo não foi selecionado, uma janela avisando o ocorrido
        # aparece, pedindo para o usuário selecionar um arquivo no campo requerido.
        
        if not arquivo_sme.get() or arquivo_sme.get() == "Selecione a planilha do SME":
            messagebox.showwarning("Aviso", "Por favor, selecione a planilha do SME.")
            return
        if not arquivo_sma.get() or arquivo_sma.get() == "Selecione a planilha do SMA":
            messagebox.showwarning("Aviso", "Por favor, selecione a planilha do SMA.")
            return
        if not arquivo_scc.get() or arquivo_scc.get() == "Selecione a planilha do SCC":
            messagebox.showwarning("Aviso", "Por favor, selecione a planilha do SCC.")
            return
        if not arquivo_ssc.get() or arquivo_ssc.get() == "Selecione a planilha do SSC":
            messagebox.showwarning("Aviso", "Por favor, selecione a planilha do SSC.")
            return
        
        # Caso não seja a base dos dados do Júpiter que está sendo feita:
        if not jupiter:
            # Verifico se os arquivos foram selecionados, e os campos preenchidos, novamente alertando se algo estiver faltando.
            if not arquivo_outros.get() or arquivo_outros.get() == "Selecione a planilha dos Outros Institutos":
                messagebox.showwarning("Aviso", "Por favor, selecione a planilha dos Outros Institutos.")
                return
            if not arquivo_salas.get() or arquivo_salas.get() == "Selecione a planilha dos dados das salas":
                messagebox.showwarning("Aviso", "Por favor, selecione a planilha dos dados das salas.")
                return
            if not ano_dados.get():
                messagebox.showwarning("Aviso", "Por favor, insira o ano dos dados.")
                return
        # Caso seja a base dos dados do Júpiter que está sendo feita:
        else:
            # Verifico se foram adicionados arquivos com os dados dos outros institutos, alertando a ausência deles.
            if len(lista_outros) == 0:
                messagebox.showwarning("Aviso", "Por favor, selecione a(s) planilha(s) dos Outros Institutos.")
                return
                
        # Faço o mesmo alerta de antes caso o usuário não tenha fornecido o nome para a nova base de dados.
        if not nome_arquivo.get():
            messagebox.showwarning("Aviso", "Por favor, insira o nome da base de dados.")
            return
        
        # Exemplo de uso das variáveis
        # print(f"Arquivo selecionado: {arquivo_selecionado.get()}")
        # print(f"Valor importante: {valor_importante.get()}")
        # concat_df(arquivo_sme.get(), arquivo_sma.get(), arquivo_scc.get(), arquivo_ssc.get(), \
        # arquivo_outros.get(), arquivo_salas.get(), nome_arquivo.get(), ano_dados.get())
        
        # Baseando-se em qual base de dados está sendo criada, chamo uma função para concatenar os dados dos arquivos fornecidos.
        if not jupiter:
            concat_df(arquivo_sme.get(), arquivo_sma.get(), arquivo_scc.get(), 
                      arquivo_ssc.get(), arquivo_salas.get(), nome_arquivo.get(), ano_dados.get(), jupiter, arquivo_outros.get())
        else:
            concat_df(arquivo_sme.get(), arquivo_sma.get(), arquivo_scc.get(), 
                      arquivo_ssc.get(), None, nome_arquivo.get(), None, jupiter, lista_outros)
        # messagebox.showinfo("Sucesso", "Valores salvos com sucesso!")
        
        # Terminada a concatenação, destruo a janela de seleção.
        nova_janela.destroy()

    # Defino um botão e seu espaço na janela para chamar a função que salva os nomes dos arquivos.
    btn_salvar = tk.Button(nova_janela, text="Criar base de dados", command=salvar_valores)
    btn_salvar.pack(pady=10)

    # Uma importante ressalva. Por conta da maneira que os botões, textos e campos podem ser colocados na janela, 
    # é possível deixar certos botões com uma ordem, mesmo que "o espaço entre eles deveria estar vazio".
    # Por exemplo: no caso de estar fazendo a base de dados do Júpiter, algumas posições podem estar denotadas como 
    # linha 7 depois de linha 5, mas o espaço onde estaria a linha 6 não possui nada. Neste caso, a linha 7 é jogada para cima,
    # preenchendo o lugar da linha 6.
    # Em outras palavras, enquanto a ordem for mantida de menor para maior, mesmo que possa "faltar" definições em determinadas linhas,
    # a ordem e espaço dos objetos e widgets será mantida.
    
    # if 
    # root2.mainloop()
    # # Checkbox 1
    # checkbox1 = Checkbutton(root, text="Opção 1", variable=var1, command=atualizar_estado)
    # checkbox1.grid(row=0, column=0, sticky="w")
    # input1 = Entry(root, state=DISABLED)  # Input associado ao Checkbox 1
    # input1.grid(row=0, column=1)

### Ler Dataframe

In [1587]:
# Função que lê um dado arquivo.
def ler_df(caminho_arquivo):
    # Tento ler um arquivo.
    try:
        # Dependendo do tipo de arquivo, precisa utilizar leituras de dados diferentes. Vale ressaltar que essa função lê apenas arquivos
        # de planilha, como '.xlsx', '.xls' e '.csv'. Com o arquivo lido, retorno o dataframe correspondente.
        if caminho_arquivo.endswith(('.xlsx', '.xls')):
            df = pd.read_excel(caminho_arquivo)
            return df
        elif caminho_arquivo.endswith('.csv'):
            df = pd.read_csv(caminho_arquivo, on_bad_lines='skip', sep=';', encoding='latin-1')
            return df
        else:
            # Caso o arquivo fornecido seja incompatível, alerto o usuário.
            raise ValueError("Formato de arquivo não suportado!")
            
    # No caso de algum erro inesperado ocorrer, alerto o usuário com uma mensagem e o nome do erro.
    except Exception as e:
            messagebox.showerror("Erro", f"Erro ao carregar o arquivo:\n{e}")
    # return df

### Concatenar

In [ ]:
# Função que concatena as planilhas em um único arquivo.
def concat_df(SME, SMA, SCC, SSC, salas, nome_arquivo, ano, jupiter, outros):
    # Crio uma variável para listar os arquivos que serão concatenados.
    files = []
    # Caso eu não esteja concatenando os arquivos para a base do JúpiterWeb:
    if not jupiter:
        # Leio a planilha com os dados das salas e salvo em um dataframe separado.
        df_salas = ler_df(salas)
        
        # Defino uma lista com os nomes dos arquivos a serem concatenados.
        name_files = [SME, SMA, SCC, SSC, outros]
        
        # Para cada nome de arquivo:
        for name in name_files:
            # Leio e salvo o arquivo em uma variável.
            df = ler_df(name)

            # Defino uma variável com o nome de um cabeçalho para ser encontrado, caso o cabeçalho não seja a primeira linha da planilha.
            header_name = 'Disciplina (código)'

            # Para cada linha e célula da primeira coluna do dataframe:
            for i, valor in enumerate(df.loc[:,df.columns[0]]):
                # Se o valor da célula for o nome do cabeçalho que estou procurando:
                if valor == header_name:
                    # Salvo o número da linha do cabeçalho.
                    header_row = i+1

                    # Chamo a função que lerá o dataframe da forma correta e o deixará no formato padronizado.
                    df_padronizado = padroniza_dataframe(name, header_row, ano)

                    # Caso o dataframe não esteja vazio, isto é, se o arquivo foi lido corretamente e não tinha nenhum erro aparente:
                    if not df_padronizado.empty:
                        # Adiciono o dataframe na lista dos que serão concatenados.
                        files.append(df_padronizado)
                    # Caso tenha ocorrido um erro, retorno para a função anterior, onde o usuário verificará os arquivos selecionados.
                    else:
                        return
                    
                    # Se o cabeçalho foi encontrado, não há motivo para continuar buscando, então encerro o loop dessa busca.
                    break
        # Com os arquivos lidos, salvo os nomes predeterminados das planilhas.
        sheets = ["SME", "SMA", "SCC", "SSC", "Outros"]

    # Para o caso de concatenando os arquivos para a base do Júpiter:
    else:
        # Defino uma lista com os nomes dos arquivos a serem concatenados.
        name_files = [SME, SMA, SCC, SSC]

        # Adiciono os nomes dos arquivos dos outros institutos a essa lista de nomes.
        name_files.extend(outros)

        # E defino uma lista para salvar a sigla das disciplinas dos outros institutos.
        other_files = []
        
        # Como anteriormente, para cada nome de arquivo:
        for name in name_files:
            # Leio e salvo o arquivo em uma variável.
            df = ler_df(name)

            # E salvo o dataframe em na lista dos arquivos que serão concatenados.
            files.append(df)

        # Para cada nome de arquivo de outros institutos:
        for name in outros:
            # Leio e salvo o arquivo em uma variável.
            df = ler_df(name)

            # Salvo a sigla das disciplinas daquele instituto. Eu considerei como sigla os 3 primeiros caracteres do nome de uma disciplina.
            # Ex: SME0230 pertence ao departamento do SME, 7600005 pertence ao departamento 760, e assim por diante.
            new_name = str(df.loc[0, 'Disciplina'])[:3]

            # Salvo essa nova sigla na lista definida anteriormente.
            other_files.append(new_name)

        # Com os arquivos lidos, salvo os nomes predeterminados das planilhas.
        sheets = ["SME", "SMA", "SCC", "SSC"]

        # E adiciono os nomes obtidos dos demais departamentos.
        sheets.extend(other_files)
            
    # Com todos os dados obtidos e padronizados, verifico se o nome da base de dados termina com '.xlsx':
    if nome_arquivo.endswith(".xlsx"):
        # Em caso positivo, deleto essa terminologia.
        nome_arquivo.replace(".xlsx", "")

    # Defino uma variável com o nome completo do arquivo (nome do arquivo + o tipo de arquivo).
    full_name = nome_arquivo + ".xlsx"

    # # Defino também um valor i, que simboliza o número de cópias do arquivo de mesmo nome.
    # i = 1
    
    # # Enquanto existir um arquivo no diretório com o nome do arquivo atual:
    # while os.path.exists(full_name):
    #     # Redefino o nome completo do arquivo, considerando como uma possível cópia.
    #     full_name = f"{nome_arquivo} ({i}){'.xlsx'}"

    #     # Incremento o sufixo até encontrar um nome disponível.
    #     i += 1

    # Com o nome disponível encontrado, atualizo o nome do arquivo para a base de dados.
    nome_arquivo = full_name

    # Para finalmente concatenar os arquivos lidos, defino o nome da base de dados com o nome disponível.
    with pd.ExcelWriter(nome_arquivo, engine="openpyxl") as writer:
        # Caso a base sendo definida não for a do Júpiter, preciso incluir a planilha de salas no arquivo.
        if not jupiter:
            df_salas.to_excel(writer, sheet_name="Salas", index=False)

        # Incluo todas as planilhas com as aulas dos departamentos no arquivo.
        for sh in range(len(files)):
            files[sh].to_excel(writer, sheet_name=sheets[sh], index=False)

    # Com o processo terminado, mostro uma mensagem confirmando que o arquivo foi criado com sucesso.
    messagebox.showinfo("", f"Arquivo {nome_arquivo} criado com sucesso!")

## Construir base de dados completa

In [ ]:
# Função que cria a base de dados completa, isto é, a que faz o cruzamento dos dados das aulas com os dados do JúpiterWeb.
# Seu parâmetro 'pior_caso' refere-se a uma variável booleana, que recebe o valor False caso seja o momento de criar a Base de Dados Completa,
# cruzando os dados das aulas com os dados do JúpiterWeb, e True caso seja o momento de criar a Base de Dados de Pior Caso.
# A Base de Dados de Pior Caso faz uma simples comparação entre os dados de dois anos distintos, misturando dados dos dois anos para
# montar uma análise com os maiores números de inscritos desses anos.
def base_dados(pior_caso):
    
    # Crio uma nova janela em cima da janela principal da interface.
    nova_janela = tk.Toplevel(root)

    # Baseado em qual base de dados será criada, defino um nome diferente para a janela.
    if not pior_caso:
        nova_janela.title("Construir Base de Dados do Modelo")

    else:
        nova_janela.title("Construir Base de Dados de Pior Caso")
    
    # Crio o frame para armazenar os botões e outros campos da nova janela.
    frame = tk.Frame(nova_janela)
    frame.pack(pady=10, padx=10)

    # Variável para armazenar o nome do arquivo
    # arquivo_ing = tk.StringVar(value="Selecione a planilha dos dados dos ingressantes")
    # Caso não esteja fazendo a base de pior caso:
    if not pior_caso:
        # Defino algumas variáveis para salvar o arquivo com a base de dados das aulas,
        # a base de dados do JúpiterWeb, e a base de dados dos ingressantes.
        arquivo_base = tk.StringVar(value="Selecione a planilha dos dados das aulas")
        arquivo_jptr = tk.StringVar(value="Selecione a planilha dos dados do júpiter")
        arquivo_ing = tk.StringVar(value="Selecione a planilha dos dados dos ingressantes")
    # Caso esteja fazendo a base de pior caso:
    else:
        # Defino as variáveis para salvar os arquivos das bases de dados.
        arquivo_base1 = tk.StringVar(value="Selecione a base de dados mais recente")
        arquivo_base2 = tk.StringVar(value="Selecione a base de dados mais antiga")

    # Defino uma variável para salvar o nome da nova base de dados, seja ela a de pior caso ou não.
    nome_arquivo = tk.StringVar()

    # Funções para selecionar um arquivo. Como a função 'mãe' (base_dados) pode ser utilizada em duas ocasiões diferentes,
    # essas funções para selecionar arquivos também são diferentes dependendo da ocasião.
    def selecionar_um():
        # Caso não esteja fazendo a base de pior caso:
        if not pior_caso:
            # O usuário seleciona o arquivo contendo a base de dados das aulas.
            arquivo = filedialog.askopenfilename(title="Selecione a planilha dos dados das aulas")
            
            # Se um arquivo foi selecionado:
            if arquivo:
                # Salvo o caminho do arquivo.
                arquivo_base.set(arquivo)
        # Caso esteja fazendo a base de pior caso:
        else:
            # O usuário seleciona o arquivo contendo a base de dados mais recente.
            arquivo = filedialog.askopenfilename(title="Selecione a base de dados mais recente")
            
            # Se um arquivo foi selecionado:
            if arquivo:
            # Salvo o caminho do arquivo.
                arquivo_base1.set(arquivo)
    def selecionar_dois():
        if not pior_caso:
            arquivo = filedialog.askopenfilename(title="Selecione a planilha dos dados do júpiter")
            if arquivo:
                arquivo_jptr.set(arquivo)
        else:
            arquivo = filedialog.askopenfilename(title="Selecione a base de dados mais antiga")
            if arquivo:
                arquivo_base2.set(arquivo)

    # Caso não esteja fazendo a base de pior caso:
    if not pior_caso:
        # Defino uma outra função para selecionar o arquivo com a base de dados dos ingressantes, com o mesmo funcionamento das demais.
        def selecionar_tres():
            arquivo = filedialog.askopenfilename(title="Selecione a planilha dos dados dos ingressantes")
            if arquivo:
                arquivo_ing.set(arquivo)
            
    # Caso não esteja fazendo a base de pior caso:
    if not pior_caso:
        # Defino legendas e botões na janela para o usuário definir os arquivos necessários.
        lbl_base = tk.Label(frame, text="Selecione a planilha dos dados das aulas")
        lbl_base.grid(row=0, column=0, pady=5)
        btn_selecionar_base = tk.Button(frame, textvariable=arquivo_base, command=selecionar_um, wraplength=250, width=40)
        btn_selecionar_base.grid(row=0, column=1, padx=5, pady=5)

        lbl_jptr = tk.Label(frame, text="Selecione a planilha dos dados do júpiter")
        lbl_jptr.grid(row=1, column=0, pady=5)
        btn_selecionar_jptr = tk.Button(frame, textvariable=arquivo_jptr, command=selecionar_dois, wraplength=250, width=40)
        btn_selecionar_jptr.grid(row=1, column=1, padx=5, pady=5)

        lbl_ing = tk.Label(frame, text="Selecione a planilha dos dados dos ingressantes")
        lbl_ing.grid(row=2, column=0, pady=5)
        btn_selecionar_ing = tk.Button(frame, textvariable=arquivo_ing, command=selecionar_tres, wraplength=250, width=40)
        btn_selecionar_ing.grid(row=2, column=1, padx=5, pady=5)

    # Caso esteja fazendo a base de pior caso:
    else:
        # Defino legendas e botões na janela para o usuário definir os arquivos necessários.
        lbl_base1 = tk.Label(frame, text="Selecione a base de dados mais recente")
        lbl_base1.grid(row=0, column=0, pady=5)
        btn_selecionar_base1 = tk.Button(frame, textvariable=arquivo_base1, command=selecionar_um, wraplength=250, width=40)
        btn_selecionar_base1.grid(row=0, column=1, padx=5, pady=5)

        lbl_base2 = tk.Label(frame, text="Selecione a base de dados mais antiga")
        lbl_base2.grid(row=1, column=0, pady=5)
        btn_selecionar_base2 = tk.Button(frame, textvariable=arquivo_base2, command=selecionar_dois, wraplength=250, width=40)
        btn_selecionar_base2.grid(row=1, column=1, padx=5, pady=5)

    # Defino a legenda e o campo na janela para o usuário inserir o nome da nova base de dados.
    lbl_arq = tk.Label(frame, text="Insira o nome para a nova base de dados:")
    lbl_arq.grid(row=3, column=0, sticky="w", pady=5)
    campo_arq = tk.Entry(frame, textvariable=nome_arquivo)
    campo_arq.grid(row=3, column=1, pady=5)

    
    # Defino uma função que salva os valores das variáveis contendo o nome dos arquivos escolhidos.
    def salvar_valores():
        # Caso não esteja fazendo a base de pior caso:
        if not pior_caso:
            # Todas as condições a seguir seguem a lógica de que, se um arquivo não foi selecionado, uma janela avisando o ocorrido
            # aparece, pedindo para o usuário selecionar um arquivo no campo requerido.
            
            if not arquivo_base.get() or arquivo_base.get() == "Selecione a planilha dos dados das aulas":
                messagebox.showwarning("Aviso", "Por favor, selecione a planilha dos dados das aulas.")
                return
            if not arquivo_jptr.get() or arquivo_jptr.get() == "Selecione a planilha dos dados do júpiter":
                messagebox.showwarning("Aviso", "Por favor, selecione a planilha dos dados do júpiter.")
                return
            if not arquivo_ing.get() or arquivo_ing.get() == "Selecione a planilha dos dados dos ingressantes":
                messagebox.showwarning("Aviso", "Por favor, selecione a planilha dos dados dos ingressantes.")
                return

        # Caso esteja fazendo a base de pior caso:
        else:
            # Todas as condições a seguir seguem a lógica de que, se um arquivo não foi selecionado, uma janela avisando o ocorrido
            # aparece, pedindo para o usuário selecionar um arquivo no campo requerido.
            if not arquivo_base1.get() or arquivo_base1.get() == "Selecione a base de dados mais recente":
                messagebox.showwarning("Aviso", "Por favor, selecione a primeira base de dados.")
                return
            if not arquivo_base2.get() or arquivo_base2.get() == "Selecione a base de dados mais antiga":
                messagebox.showwarning("Aviso", "Por favor, selecione a segunda base de dados.")
                return

        # Crio uma variável auxiliar para salvar o nome da nova base de dados dada pelo usuário.
        nome = nome_arquivo.get()

        # Se o usuário não forneceu um nome para a nova base de dados:
        if not nome:
            # Uma janela alertando o ocorrido é aberta.
            messagebox.showwarning("Aviso", "Por favor, insira o nome da nova base de dados.")
            return
        
        # Como anteriormente, essa seção do código serve para identificar se o nome da nova base de dados já existe,
        # criando uma cópia se já houver um arquivo com o nome fornecido.
        if nome.endswith(".xlsx"):
            nome.replace(".xlsx", "")

        full_name = nome + ".xlsx"
        # i = 1
        
        # while os.path.exists(full_name):
        #     full_name = f"{nome} ({i}){'.xlsx'}"
        #     i += 1

        nome = full_name
        
        # Caso não esteja fazendo a base de pior caso:
        if not pior_caso:
            # Tento executar o script do código responsável em cruzar os dados das aulas com os do Júpiter.
            try:
                file1 = arquivo_base.get()
                file2 = arquivo_jptr.get()
                file3 = arquivo_ing.get()
                # subprocess(["python", "jupiter sheet maker.py", [df_filename1, df_filename2, df_filename3]])
                subprocess.run(["python", "jupiter sheet maker.py", file1, file2, file3, nome])

                # Com o processo terminado, mostro uma mensagem confirmando que o arquivo foi criado com sucesso.
                messagebox.showinfo("Sucesso!", f"Base de dados para o modelo criada com sucesso. Verifique o arquivo {nome}.")

            # Caso ocorra algum erro durante a execução do script, uma janela alertando o erro é apresentada para o usuário.
            except subprocess.CalledProcessError as e:
                messagebox.showerror("Erro", f"Erro ao executar o script:\n{e.stderr}")

            # Caso um erro inesperado ocorra, uma janela alertando o erro é apresentada para o usuário.
            except Exception as e:
                messagebox.showerror("Erro", f"Erro inesperado: {e}")

        # Caso esteja fazendo a base de pior caso:
        else:
            # sheets = ["SME", "SMA", "SCC", "SSC"]
            # Leio os dois dataframes fornecidos pelo usuário.
            df1 = pd.read_excel(arquivo_base1.get())
            df2 = pd.read_excel(arquivo_base2.get())

            # Chamo a função que faz um dataframe com o maior número de inscritos de cada ano.
            df_pior_caso = base_pior_caso(df1, df2)

            # Com a base de pior caso feita, salvo-a em um arquivo de Excel com o nome fornecido pelo usuário.
            with pd.ExcelWriter(nome, engine="openpyxl") as writer:
                for sh, df_sh in df_pior_caso.items():
                    df_sh.to_excel(writer, sheet_name=sh, index=False)

            # Com o processo terminado, mostro uma mensagem confirmando que o arquivo foi criado com sucesso.
            messagebox.showinfo("", f"Arquivo {nome} criado com sucesso!")

        # messagebox.showinfo("Sucesso", "Valores salvos com sucesso!")
        # Terminada a criação da base de dados, destruo a janela de seleção.
        nova_janela.destroy()

    # Defino um botão e seu espaço na janela para chamar a função que salva os nomes dos arquivos.
    btn_salvar = tk.Button(nova_janela, text="Criar base de dados", command=salvar_valores)
    btn_salvar.pack(pady=10)

    # root2.mainloop()
    # # Checkbox 1
    # checkbox1 = Checkbutton(root, text="Opção 1", variable=var1, command=atualizar_estado)
    # checkbox1.grid(row=0, column=0, sticky="w")
    # input1 = Entry(root, state=DISABLED)  # Input associado ao Checkbox 1
    # input1.grid(row=0, column=1)

### Pior caso

In [1590]:
# Função que faz a análise de pior caso.
# Note que df1 deve sempre ser a base de dados mais recente, enquanto df2 deve sempre ser a base de dados mais antiga.
def base_pior_caso(df1, df2):
    
    # df1 = pd.read_excel(df1_name, sheet_name=sheets)
    # df2 = pd.read_excel(df1_name, sheet_name=sheets)
    # Para cada planilha na base de dados mais recente:
    for sh in df1.sheet_names[1:]:
        # Considerando que df1 e df2 possuem as mesmas planilhas (ou, pelo menos, deveriam), crio duas variáveis auxiliares
        # para salvar os dados a mesma planilha de ambas as bases.
        df1_sh = df1[sh]
        df2_sh = df2[sh]

        # Crio também duas variáveis para salvar as colunas de disciplinas das bases como listas.
        l1 = df1_sh['Disciplina (código)'].tolist()
        l2 = df2_sh['Disciplina (código)'].tolist()

        # Para cada disciplina 'd' na lista de disciplinas da base de dados mais recente:
        for d in l1:
            # Verifico se 'd' está na lista de disciplinas da base de dados mais antiga:
            if d in l2:
                # Em caso positivo, verifico se o número de inscritos do ano mais recente da disciplina é menor
                # que o número de inscritos do ano mais antigo da disciplina.
                if df1[sh].loc[l1.index(d), 'Vagas por disciplina'] < df2[sh].loc[l2.index(d), 'Vagas por disciplina']:
                    # Se for o caso, então no ano anterior houveram mais inscritos, isto é, no ano mais recente, pode ser que
                    # o número de inscritos chegue até esse valor ou o ultrapasse.
                    # Por conta disso, atualizo o número de inscritos e o ano dos dados da disciplina 'd' na base mais recente.
                    df1[sh].loc[l1.index(d), 'Vagas por disciplina'] = df2[sh].loc[l2.index(d), 'Vagas por disciplina']
                    df1[sh].loc[l1.index(d), 'Ano dos dados'] = df2[sh].loc[l2.index(d), 'Ano dos dados']

    # Após concluir a análise, retorno a base de dados atualizada.
    return df1

        

## Executar

In [1591]:
# Função que faz a verificação de dados e execução do modelo.
def execute():
    # Crio uma nova janela em cima da janela principal da interface.
    nova_janela = tk.Toplevel(root)
    nova_janela.geometry("+200+200")
    nova_janela.title("Verificar Dados e Executar Modelo")
    
    # Crio o frame para armazenar os botões e outros campos da nova janela.
    frame = tk.Frame(nova_janela)
    frame.pack(pady=10, padx=10)

    # Defino uma variável para salvar o arquivo com a base de dados para o modelo, seja ela a completa ou a de pior caso.
    arquivo_selecionado1 = tk.StringVar(value="Selecione uma base de dados")
    
    # Defino a função para selecionar a base de dados.
    def selecionar_arquivo1():
        # O usuário seleciona o arquivo contendo a base de dados das aulas.
        arquivo = filedialog.askopenfilename(title="Selecione uma base de dados")

        # Se um arquivo foi selecionado:
        if arquivo:
            # Salvo o caminho do arquivo.
            arquivo_selecionado1.set(arquivo)

    

    # Defino um botão, e sua posição na janela, para o usuário escolher a base de dados.
    btn_selecionar1 = tk.Button(frame, textvariable=arquivo_selecionado1, 
                                command=selecionar_arquivo1, wraplength=250, width=40)
    btn_selecionar1.grid(row=0, column=0, padx=5, pady=5)

    
    # Defino uma linha de separação horizontal na janela, separando o botão de seleção de arquivo do botão de verificação de dados.
    vd_separator = ttk.Separator(frame, orient="horizontal")
    vd_separator.grid(row=2, column=0, sticky="ew", pady=(0, 10))

    # Defino um botão, e sua posição na janela, para o usuário abrir o menu de verificação de dados.
    vd1 = ttk.Button(frame, text="Selecionar Verificação de Dados", 
                     command=lambda: Novo_edit_config(file_name=arquivo_selecionado1.get(), verify=True))
    vd1.grid(row=4, column=0, pady=5)

    # Defino uma linha de separação horizontal na janela, separando o botão de verificação de dados do botão de execução do modelo.
    em_separator = ttk.Separator(frame, orient="horizontal")
    em_separator.grid(row=5, column=0, sticky="ew", pady=(0,10))

    # Defino um botão, e sua posição na janela, para o usuário abrir o menu de execução do modelo.
    em = ttk.Button(frame, text="Selecionar Modelo", command=lambda: Novo_edit_config(file_name=arquivo_selecionado1.get(), verify=False))
    em.grid(row=6, column=0, pady=5)

    
            
    # Parte 1
    # parte1_label = ttk.Label(main_frame, text="Parte 1", font=("Arial", 12, "bold"))
    # parte1_label.grid(row=0, column=0, sticky="w", pady=(0, 5))
    
    # separator1 = ttk.Separator(main_frame, orient="horizontal")
    # separator1.grid(row=1, column=0, sticky="ew", pady=(0, 10))
    
    # botao_parte1 = tk.Button(main_frame, text="Botão da Parte 1")
    # botao_parte1.grid(row=2, column=0, pady=5)

### Roda Script

In [1592]:
# Função que roda um script em Python. Seus parâmetros são variáveis fornecidas pelo usuário, como qual script
# o nome da base de dados que será lida, e o peso das variáveis do modelo.
def roda_script(script, nome, peso_x, peso_y, peso_v, peso_z, alpha, qtd_pos, pref):
    
    # # Faço uma breve verificação para saber se foi fornecida uma base de dados.
    # if nome == "Selecione uma base de dados":
    #     # Se nenhum arquivo foi selecionado, uma janela avisando o ocorrido
    #     # aparece, pedindo para o usuário fornecer uma base de dados.
    #     messagebox.showwarning("Aviso", "Por favor, insira o nome da nova base de dados.")
    #     return
        
    # Crio uma janela para mostrar a saída do script, como prints importantes e até saídas de erros.
    output_window = tk.Toplevel(root)
    output_window.title("Saída do Script")
    output_text = scrolledtext.ScrolledText(output_window, width=120, height=40)
    output_text.pack()

    # Tento executar o script escolhido pelo usuário.
    try:
        # Executo o script utilizando subprocess.
        process = subprocess.Popen(
            ["python", script, nome, peso_x, peso_y, peso_v, peso_z, alpha, qtd_pos, pref],  
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        
        # Capturo e exibo a saída linha por linha do script.
        # Para cada linha da saída:
        for line in iter(process.stdout.readline, ""):
            # Verifico se a linha está vazia, isto é, se cheguei no final da saída do script.
            if line.strip():
                # Se não cheguei, insiro a linha no final da janela.
                output_text.insert(tk.END, line)

                # Também coloco para o rodapé rolar automaticamente até o final da janela.
                output_text.see(tk.END)

                # Por fim, atualizo a interface para que o usuário veja a linha mais recente da saída.
                root.update()

        # Se algum erro ocorrer durante a execução do script, ele é enviado para a variável stderr, e não é mostrado pela seção anterior.
        # Assim, salvo o possível erro do script na variável stderr.
        stderr = process.communicate()[1]

        # Se ocorreu um erro, isto é, se stderr não é uma variável vazia:
        if stderr:
            # Insiro a linha de erro no final da janela.
            output_text.insert(tk.END, "\n[ERRO]:\n" + stderr)

            # E rolo automaticamente até o final da janela.
            output_text.see(tk.END)

    # Caso algum erro ocorra ao executar o script, alerto o usuário inserindo o erro na janela de saída.
    except Exception as e:
        output_text.insert(tk.END, f"\nErro ao executar o script: {e}")

# def roda_script2(script, arquivo_selecionado1, valor):
#     nome = arquivo_selecionado1
#     if nome == "Selecione uma base de dados":
#         messagebox.showwarning("Aviso", "Por favor, insira o nome da nova base de dados.")
#         return
#     # Criar a janela para exibir a saída
#     output_window = tk.Toplevel(root)
#     output_window.title("Saída do Script")
#     output_text = scrolledtext.ScrolledText(output_window, width=120, height=40)
#     output_text.pack()
#     try:
#         # Executar o script como subprocess
#         process = subprocess.Popen(
#             ["python", script, nome, string(valor)],  # SUBSTITUIR Modelo 1.py
#             stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
#         )
        
#         # Capturar e exibir a saída linha por linha
#         for line in iter(process.stdout.readline, ""):
#             if line.strip():
#                 output_text.insert(tk.END, line)
#                 output_text.see(tk.END)  # Rolar automaticamente até o final
#                 root.update()  # Atualizar a interface

#         # Exibir erros, se houver
#         stderr = process.communicate()[1]
#         if stderr:
#             output_text.insert(tk.END, "\n[ERRO]:\n" + stderr)
#             output_text.see(tk.END)

#     except Exception as e:
#         output_text.insert(tk.END, f"\nErro ao executar o script: {e}")

### Novo Edit Configs

In [1593]:
# Função que cria uma nova janela para configurar os parâmetros da verificação ou do modelo antes de chamar o script.
# Seus parâmetros são o nome da base de dados (file_name) e uma variável booleana 'verify', que recebe o valor True quando
# o menu é de configurações da Verificação dos Dados, e False quando o menu é de configurações para Executar o Modelo
def Novo_edit_config(file_name, verify):
    
    # Faço uma breve verificação para saber se foi fornecida uma base de dados.
    if file_name == "Selecione uma base de dados":
        # Se nenhum arquivo foi selecionado, uma janela avisando o ocorrido
        # aparece, pedindo para o usuário fornecer uma base de dados.
        messagebox.showwarning("Aviso", "Por favor, selecione uma base de dados.")
        return

    # print(root.geometry)
    # Crio uma nova janela em cima da janela principal da interface.
    configs = tk.Toplevel(root)
    configs.geometry("+360+150")
    configs.title("Personalizar parâmetros")
    
    # Crio o frame para armazenar os botões e outros campos da nova janela.
    frame2 = tk.Frame(configs)
    frame2.pack(pady=10, padx=10)

    # Defino 'casais' de variáveis. Cada variável refere-se a um valor de um checkbox de uma linha.
    # Ex: var11 refere-se ao valor do checkbox da primeira linha e da primeira coluna, enquanto
    # var22 refere-se ao valor do checkbox da primeira linha e da segunda coluna.
    var11 = IntVar()
    var12 = IntVar()
    var21 = IntVar()
    var22 = IntVar()
    var31 = IntVar()
    var32 = IntVar()
    var41 = IntVar()
    var42 = IntVar()
    var51 = IntVar()
    var52 = IntVar()
    var61 = IntVar()
    var62 = IntVar()
    var71 = IntVar()
    var72 = IntVar()

    # Defino uma função que atualiza o estado de cada checkbox após marcar, ou desmarcar, um deles.
    def atualizar_estado():
        # Caso o menu não seja o de configurar a verificação dos dados:
        if not verify:
            # Se o checkbox da primeira linha e primeira coluna estiver marcado:
            if var11.get():
                # Desabilito o checkbox da primeira linha e segunda coluna.
                checkbox12.config(state=DISABLED)
            # Se ele não estiver mais marcado:
            else:
                # Retorno o checkbox da primeira linha e segunda coluna ao normal.
                checkbox12.config(state=NORMAL)
            # Se o checkbox da primeira linha e segunda coluna estiver marcado:
            if var12.get():
                # Desabilito o checkbox da primeira linha e segunda coluna.
                checkbox11.config(state=DISABLED)
                # E habilito o campo correspondente para o usuário inserir um valor.
                peso_x.config(state=NORMAL)
            # Se ele não estiver mais marcado:
            else:
                # Retorno o checkbox da primeira linha e primeira coluna ao normal.
                checkbox11.config(state=NORMAL)
                # E deleto o conteúdo inserido no campo, além de desabilitá-lo novamente.
                peso_x.delete(0, "end")
                peso_x.config(state=DISABLED)

            # As demais condições são análogas às anteriores.
            
            if var21.get():
                checkbox22.config(state=DISABLED)
            else:
                checkbox22.config(state=NORMAL)
            if var22.get():
                checkbox21.config(state=DISABLED)
                peso_y.config(state=NORMAL)
            else:
                checkbox21.config(state=NORMAL)
                peso_y.delete(0, "end")
                peso_y.config(state=DISABLED)
            
            if var31.get():
                checkbox32.config(state=DISABLED)
            else:
                checkbox32.config(state=NORMAL)
            if var32.get():
                checkbox31.config(state=DISABLED)
                peso_v.config(state=NORMAL)
            else:
                checkbox31.config(state=NORMAL)
                peso_v.delete(0, "end")
                peso_v.config(state=DISABLED)
            
            if var41.get():
                checkbox42.config(state=DISABLED)
            else:
                checkbox42.config(state=NORMAL)
            if var42.get():
                checkbox41.config(state=DISABLED)
                peso_z.config(state=NORMAL)
            else:
                checkbox41.config(state=NORMAL)
                peso_z.delete(0, "end")
                peso_z.config(state=DISABLED)
                
            if var51.get():
                checkbox52.config(state=DISABLED)
            else:
                checkbox52.config(state=NORMAL)
            if var52.get():
                checkbox51.config(state=DISABLED)
                alpha.config(state=NORMAL)
            else:
                checkbox51.config(state=NORMAL)
                alpha.delete(0, "end")
                alpha.config(state=DISABLED)
                
            if var61.get():
                checkbox62.config(state=DISABLED)
            else:
                checkbox62.config(state=NORMAL)
            if var62.get():
                checkbox61.config(state=DISABLED)
                pref.config(state=NORMAL)
            else:
                checkbox61.config(state=NORMAL)
                pref.delete(0, "end")
                pref.config(state=DISABLED)

        # Esse par de variáveis não está separado com as demais por elas serem associadas com os únicos valores necessários
        # tanto para a verificação dos dados, quanto para a execução do modelo, que são o número de alunos da pós a serem
        # considerados em disciplinas com espelho.
        if var71.get():
            checkbox72.config(state=DISABLED)
        else:
            checkbox72.config(state=NORMAL)
        if var72.get():
            checkbox71.config(state=DISABLED)
            qtd_pos.config(state=NORMAL)
        else:
            checkbox71.config(state=NORMAL)
            qtd_pos.delete(0, "end")
            qtd_pos.config(state=DISABLED)

    # Caso o menu não seja o de configurar a verificação dos dados:
    if not verify:
        # Defino uma linha puramente estética no topo da janela.
        # Em seguida, defino uma legenda e um botão ao lado, ambos desabilitados, apenas para servir como
        # instruções e mostrar as configurações recomendadas dos parâmetros.
        ttk.Separator(frame2, orient="horizontal").grid(row=0, column=0, columnspan=3, sticky="ew", pady=(0, 10))
        lbl = tk.Label(frame2, text="Selecione e/ou digite os valores que deseja para os parâmetros.", state=DISABLED)
        lbl.grid(row=1,column=0,sticky='w')
        btnl = ttk.Button(frame2, text="Recomendações - Configurações Recomendadas", state=DISABLED)
        btnl.grid(row=1,column=1,sticky='e')
        Tooltip(btnl, "Recomendado:\n\
        Peso de alocação: 1\n\
        Peso de troca de sala: 500\n\
        Peso de agrupamento: 1\n\
        Peso de superlotação: 10\n\
        Índice de superlotação: 0.85\n\
        Peso de salas preferencialmente vazias: 1\n\
        Quantidade de alunos da pós: 20")

        # Semelhantemente, os próximos 'blocos de código' seguem a mesma lógica.
        # Crio uma linha para separar o espaço de cada parâmetro.
        # Incluo uma legenda com uma Tooltip para explicar o propósito do parâmetro.
        # Crio uma checkbox para marcar a configuração padrão daquele parâmetro sempre primeiro.
        # Crio, após ele, um checkbox para marcar a configuração personalizada com um campo desabilitado ao lado dele.
        # Ele precisa estar desabilitado inicialmente, ou é possível inserir um valor mesmo sem marcar um checkbox.
        ttk.Separator(frame2, orient="horizontal").grid(row=2, column=0, columnspan=3, sticky="ew", pady=(0, 10))
        lblx = tk.Label(frame2, text="Defina um peso de alocação:")
        lblx.grid(row=3, column=0, sticky='w', pady=5)
        # checkbox1 = Checkbutton(frame1, text="Modelo 1", variable=var1, command=atualizar_estado)
        checkbox11 = Checkbutton(frame2, text="Recomendado: 1", variable=var11, command=atualizar_estado)
        checkbox11.grid(row=3, column=1, sticky='w', pady=5)
        checkbox12 = Checkbutton(frame2, text="Personalizado: ", variable=var12, command=atualizar_estado)
        checkbox12.grid(row=3, column=2, sticky='w', pady=5)
        peso_x = Entry(frame2, state=DISABLED)
        peso_x.grid(row=4, column=2, sticky='e', pady=5)
        Tooltip(lblx, "O peso de alocação refere-se ao peso da variável x_as no modelo. \n\
        Quanto mais alto, mais importante será essa variável,\n\
        logo, o modelo pode deixar de alocar de forma ótima se o peso for baixo.\n\
        Por exemplo, ele pode entender que é mais importante agrupar \n\
        os cursos, do que usar o maior espaço possível.")
        
        ttk.Separator(frame2, orient="horizontal").grid(row=5, column=0, columnspan=3, sticky="ew", pady=(0, 10))
        lbly = tk.Label(frame2, text="Defina um peso de troca de sala:")
        lbly.grid(row=6,column=0,sticky='w',pady=5)
        checkbox21 = Checkbutton(frame2, text="Recomendado: 500", variable=var21, command=atualizar_estado)
        checkbox21.grid(row=6, column=1, sticky='w', pady=5)
        checkbox22 = Checkbutton(frame2, text="Personalizado: ", variable=var22, command=atualizar_estado)
        checkbox22.grid(row=6, column=2, sticky='w', pady=5)
        peso_y = Entry(frame2, state=DISABLED)
        peso_y.grid(row=7,column=2,sticky='e',pady=5)
        Tooltip(lbly, "O peso de troca de sala refere-se ao peso da variável y_t no modelo. \n\
        Quanto mais alto, mais importante será essa variável,\n\
        logo, o modelo pode deixar de manter aulas de uma \n\
        turma/disciplina alocadas na mesma sala se o peso for baixo.\n\
        Por exemplo, ele pode entender que é mais importante preencher \n\
        uma sala o máximo possível sem verificar quais são elas.")

    
        ttk.Separator(frame2, orient="horizontal").grid(row=8, column=0, columnspan=3, sticky="ew", pady=(0, 10))
        lblv = tk.Label(frame2, text="Defina um peso de agrupamento:")
        lblv.grid(row=9,column=0,sticky='w',pady=5)
        checkbox31 = Checkbutton(frame2, text="Recomendado: 1", variable=var31, command=atualizar_estado)
        checkbox31.grid(row=9, column=1, sticky='w', pady=5)
        checkbox32 = Checkbutton(frame2, text="Personalizado: ", variable=var32, command=atualizar_estado)
        checkbox32.grid(row=9, column=2, sticky='w', pady=5)
        peso_v = Entry(frame2, state=DISABLED)
        peso_v.grid(row=10,column=2,sticky='e',pady=5)
        Tooltip(lblv, "O peso de agrupamento refere-se ao peso da variável v_cssl no modelo. \n\
        Quanto mais alto, mais importante será essa variável,\n\
        logo, o modelo pode deixar de tentar agrupar as aulas de \n\
        um curso próximas umas das outras se o peso for baixo.\n\
        Por exemplo, ele pode entender que é mais importante \n\
        acomodar os alunos numa sala, do que agrupar os cursos.")
    
        ttk.Separator(frame2, orient="horizontal").grid(row=11, column=0, columnspan=3, sticky="ew", pady=(0, 10))
        lblz = tk.Label(frame2, text="Defina um peso de superlotação:")
        lblz.grid(row=12,column=0,sticky='w',pady=5)
        checkbox41 = Checkbutton(frame2, text="Recomendado: 10", variable=var41, command=atualizar_estado)
        checkbox41.grid(row=12, column=1, sticky='w', pady=5)
        checkbox42 = Checkbutton(frame2, text="Personalizado: ", variable=var42, command=atualizar_estado)
        checkbox42.grid(row=12, column=2, sticky='w', pady=5)
        peso_z = Entry(frame2, state=DISABLED)
        peso_z.grid(row=13,column=2,sticky='e',pady=5)
        Tooltip(lblz, "O peso de superlotação refere-se ao peso da variável z_as no modelo. \n\
        Quanto mais alto, mais importante será essa variável,\n\
        logo, o modelo pode deixar de tentar acomodar melhor os \n\
        alunos em uma sala se o peso for baixo.\n\
        Por exemplo, ele pode entender que é mais importante \n\
        preencher uma sala o máximo possível, do que deixar alguns lugares desocupados \n\
        para acomodar melhor os alunos, ou para receber alunos vindos do requerimento.")
    
        ttk.Separator(frame2, orient="horizontal").grid(row=14, column=0, columnspan=3, sticky="ew", pady=(0, 10))
        lbla = tk.Label(frame2, text="Defina um índice de superlotação (valor entre 0 e 1, utilizando '.'):")
        lbla.grid(row=15,column=0,sticky='w',pady=5)
        checkbox51 = Checkbutton(frame2, text="Recomendado: 0.85", variable=var51, command=atualizar_estado)
        checkbox51.grid(row=15, column=1, sticky='w', pady=5)
        checkbox52 = Checkbutton(frame2, text="Personalizado: ", variable=var52, command=atualizar_estado)
        checkbox52.grid(row=15, column=2, sticky='w', pady=5)
        alpha = Entry(frame2, state=DISABLED)
        alpha.grid(row=16,column=2,sticky='e',pady=5)
        Tooltip(lbla, "O índice de superlotação refere-se à porcentagem usada pela variável z_as no modelo. \n\
        Em outras palavras, é o quanto uma aula preenche uma sala \n\
        para que o modelo tente colocar essa aula em uma sala maior, \n\
        garantindo uma melhor acomodação para os alunos e espaço para \n\
        alunos vindos do requerimento. Quanto mais alto, mais o modelo \n\
        prioriza preencher o máximo possível de uma sala. Um valor baixo \n\
        faz com que as salas maiores sejam preenchidas com mais facilidade, \n\
        e um valor alto fará o mesmo, mas para as salas menores.")
    
        ttk.Separator(frame2, orient="horizontal").grid(row=17, column=0, columnspan=3, sticky="ew", pady=(0, 10))
        lbl_pref = tk.Label(frame2, text="Defina um peso de salas preferencialmente vazias:")
        lbl_pref.grid(row=18,column=0,sticky='w',pady=5)
        checkbox61 = Checkbutton(frame2, text="Recomendado: 500", variable=var61, command=atualizar_estado)
        checkbox61.grid(row=18, column=1, sticky='w', pady=5)
        checkbox62 = Checkbutton(frame2, text="Personalizado: ", variable=var62, command=atualizar_estado)
        checkbox62.grid(row=18, column=2, sticky='w', pady=5)
        pref = Entry(frame2, state=DISABLED)
        pref.grid(row=19,column=2,sticky='e',pady=5)
        Tooltip(lbl_pref, "O peso de salas preferencialmente vazias refere-se à importância de determinadas salas ficarem desocupadas no modelo. \n\
        Quanto mais alto, maior a chance de essas salas não serem utilizadas,\n\
        logo, o modelo pode tentar alocar aulas em salas indesejadas \n\
        se o peso for baixo.\n\
        Por exemplo, ele pode entender que é mais importante \n\
        preencher uma sala que poderia ser usada para alguma outra função, por ela ser mais fácil de ser preenchida, \n\
        do que mantê-la desocupada para a pós.")

    ttk.Separator(frame2, orient="horizontal").grid(row=20, column=0, columnspan=3, sticky="ew", pady=(0, 10))
    lblq = tk.Label(frame2, text="Defina uma quantidade de alunos da pós para as disciplinas com espelho:")
    lblq.grid(row=21,column=0,sticky='w',pady=5)
    checkbox71 = Checkbutton(frame2, text="Recomendado: 20", variable=var71, command=atualizar_estado)
    checkbox71.grid(row=21, column=1, sticky='w', pady=5)
    checkbox72 = Checkbutton(frame2, text="Personalizado: ", variable=var72, command=atualizar_estado)
    checkbox72.grid(row=21, column=2, sticky='w', pady=5)
    qtd_pos = Entry(frame2, state=DISABLED)
    qtd_pos.grid(row=22,column=2,sticky='e',pady=5)
    Tooltip(lblq, "A quantidade de alunos da pós refere-se a um valor sugerido pelo usuário \n\
    para simbolizar quantos alunos da pós-graduação farão \n\
    uma disciplina da graduação que é espelhada com a pós. \n\
    Esse valor será aplicado no número de inscritos de QUALQUER \n\
    disciplina que tenha espelho com a pós.")

    # Defino uma função que detecta as configurações escolhidas pelo usuário.
    def executar_configs_perso():
        # Caso o menu não seja o de configurar a verificação dos dados:
        if not verify:
            # Verifico se ao menos uma das checkbox relacionadas ao peso de alocação foi selecionada,
            # pois o modelo precisa desse peso obrigatoriamente.
            if not var11.get() and not var12.get():
                # Se nenhum peso foi fornecido, uma janela alerta o ocorrido e pede para o usuário definir um peso.
                messagebox.showwarning("Aviso", "Por favor, defina um peso de alocação.")
                return

            # Verifico se a checkbox da configuração recomendada está marcado:
            elif var11.get():
                # Se estiver, dou o valor recomendado para a variável auxiliar.
                aux_x = '1'

            # Verifico se a checkbox da configuração personalizada está marcado:
            else:
                # Se estiver, dou o valor fornecido pelo usuário para a variável auxiliar.
                aux_x = peso_x.get()

            # Verifico se ao menos uma das checkbox relacionadas ao peso de troca de sala foi selecionada,
            # pois o modelo precisa desse peso obrigatoriamente. A lógica é a mesma da de anteriormente.
            if not var21.get() and not var22.get():
                messagebox.showwarning("Aviso", "Por favor, defina um peso de troca de sala.")
                return
            elif var21.get():
                aux_y = '500'
            else:
                aux_y = peso_y.get()

            # As condições a seguir seguem a mesma lógica para cada parâmetro e configuração disponível.
            # Verifico se a checkbox da configuração personalizada está marcada e sem um valor inserido pelo usuário:
            if var12.get() and not peso_x.get():
                # Se for o caso, uma janela alerta o ocorrido e pede para o usuário definir um valor para o parâmetro.
                messagebox.showwarning("Aviso", "Por favor, digite um peso de alocação.")
                return

            # Verifico se a checkbox da configuração recomendada está marcada:
            elif var11.get():
                # Se estiver, dou o valor recomendado para a variável auxiliar.
                aux_x = '1'

            # Verifico, por fim, se a checkbox da configuração personalizada está marcada.
            else:
                # Se estiver, para ter entrado nessa condição, o usuário deve ter fornecido um valor para o parâmetro.
                # Portanto, dou o valor fornecido para a variável auxiliar.
                aux_x = peso_x.get()
                
            if var22.get() and not peso_y.get():
                messagebox.showwarning("Aviso", "Por favor, digite um peso de troca de sala.")
                return
            elif var21.get():
                aux_y = '500'
            else:
                aux_y = peso_y.get()
                
            if var32.get() and not peso_v.get():
                messagebox.showwarning("Aviso", "Por favor, digite um peso de agrupamento.")
                return
            elif var31.get():
                aux_v = '1'
            else:
                aux_v = peso_v.get()
                
            if var42.get() and not peso_z.get():
                messagebox.showwarning("Aviso", "Por favor, digite um peso de superlotação.")
                return
            elif var41.get():
                aux_z = '10'
            else:
                aux_z = peso_z.get()
                
            if var52.get() and not alpha.get():
                messagebox.showwarning("Aviso", "Por favor, digite um índice de superlotação.")
                return
            elif var51.get():
                aux_a = '0.85'
            else:
                aux_a = alpha.get()
                
            if var62.get() and not pref.get():
                messagebox.showwarning("Aviso", "Por favor, digite um peso de salas preferencialmente vazias.")
                return
            elif var61.get():
                aux_p = '500'
            else:
                aux_p = pref.get()

            if var72.get() and not qtd_pos.get():
                messagebox.showwarning("Aviso", "Por favor, digite uma quantidade de alunos da pós para as disciplinas com espelho.")
                return
            elif var71.get():
                aux_q = '20'
            else:
                aux_q = qtd_pos.get()
            # else:
            #     aux_q = '0'

            # Se nenhum alerta foi gerado, as configurações personalizadas devem estar corretas, 
            # então só resta rodar o script com elas.
            roda_script(script="Modelo Universal-Copy1.py", 
                    nome=file_name, peso_x=aux_x, peso_y=aux_y, peso_v=aux_v, peso_z=aux_z, 
                    alpha=aux_a, qtd_pos=aux_q, pref=aux_p)

        # Caso o menu seja o de configurar a verificação dos dados:
        else:
            # Faço apenas a verificação das checkbox referentes à quantida de aluno da pós das disciplinas espelho, como anteriormente.
            if var72.get() and not qtd_pos.get():
                messagebox.showwarning("Aviso", "Por favor, digite uma quantidade de alunos da pós para as disciplinas com espelho.")
                return
            elif var71.get():
                aux_q = '20'
            else:
                aux_q = qtd_pos.get()
            # else:
            #     aux_q = '0'

            # Após isso, rodo o script com as configurações necessárias.
            roda_script(script="verificar_dados.py", 
                    nome=file_name, peso_x="", peso_y="", peso_v="", peso_z="", 
                    alpha="", qtd_pos=aux_q, pref="")
            
       

    # Caso o menu não seja o de configurar a verificação dos dados:
    if not verify:
        # Crio uma linha horizontal para separar as escolhas de parâmetro dos botões da janela.
        ttk.Separator(frame2, orient="horizontal").grid(row=23, column=0, columnspan=2, sticky="w", pady=(0, 10))

        # Defino o botão de Executar o Modelo com os parâmetros Recomendados, que chama a função de rodar script e os 
        # parâmetros recomendados diretamente.
        ttk.Button(frame2, text="Executar com Recomendados", 
                   command=lambda: roda_script(script="Modelo Universal-Copy1.py", 
                                               nome=file_name, peso_x='1', peso_y='500', peso_v="", 
                                               peso_z='10', alpha='0.85', qtd_pos='20', pref='500')).grid(row=24, column=0, sticky='w', pady=5)

        # Defino o botão de Executar o Modelo com os parâmetros Personalizados, que chama a função definida anteriormente
        # para checar as configurações personalizadas pelo usuário.
        ttk.Button(frame2, text="Executar com Personalizados", 
                   command=executar_configs_perso).grid(row=24, column=2, sticky='w', pady=5)

    # Caso o menu seja o de configurar a verificação dos dados:
    else:
        # Análogo aos botões que são definidos no menu de configurações do modelo.
        ttk.Separator(frame2, orient="horizontal").grid(row=23, column=0, columnspan=2, sticky="w", pady=(0, 10))
        ttk.Button(frame2, text="Verificar com Recomendados", 
                   command=lambda: roda_script(script="verificar_dados.py", 
                                               nome=file_name, peso_x='1', peso_y='500', peso_v="", 
                                               peso_z='10', alpha='0.85', qtd_pos='20', pref='500')).grid(row=24, column=0, sticky='w', pady=5)
    
        ttk.Button(frame2, text="Verificar com Personalizados", 
                   command=executar_configs_perso).grid(row=24, column=2, sticky='w', pady=5)
    # roda_script(script, nome, peso_x, peso_y, peso_v, peso_z, alpha, qtd_pos)
    

### Menu dos modelos (obsoleto)

#### Antigo menu

In [1594]:
# Variáveis associadas aos checkboxes
def menu_modelo(nome):
    # nome = file_name.get()
    if nome == "Selecione uma base de dados":
        messagebox.showwarning("Aviso", "Por favor, selecione uma base de dados.")
        return
    menu = tk.Toplevel(root)
    menu.geometry("+550+250")
    menu.title("Selecionar Modelo")
    
    # Frame para o botão e o campo
    frame1 = tk.Frame(menu)
    frame1.pack(pady=10, padx=10)
    
    # name = file_name.get()
    var1 = IntVar()
    var2 = IntVar()
    var3 = IntVar()
    var4 = IntVar()
    
    def atualizar_estado():
        # Se qualquer checkbox estiver marcado, desabilite os outros
        if var1.get() or var2.get() or var3.get() or var4.get():
            # Desabilita os checkboxes não selecionados
            checkbox1.config(state=DISABLED if not var1.get() else NORMAL)
            btn11.config(state=DISABLED if not var1.get() else NORMAL)
            btn12.config(state=DISABLED if not var1.get() else NORMAL)
            checkbox2.config(state=DISABLED if not var2.get() else NORMAL)
            btn21.config(state=DISABLED if not var2.get() else NORMAL)
            btn22.config(state=DISABLED if not var2.get() else NORMAL)
            checkbox3.config(state=DISABLED if not var3.get() else NORMAL)
            btn31.config(state=DISABLED if not var3.get() else NORMAL)
            btn32.config(state=DISABLED if not var3.get() else NORMAL)
            checkbox4.config(state=DISABLED if not var4.get() else NORMAL)
            btn41.config(state=DISABLED if not var4.get() else NORMAL)
            btn42.config(state=DISABLED if not var4.get() else NORMAL)
            # checkbox3.config(state=DISABLED if not var3.get() else NORMAL)
            # checkbox4.config(state=DISABLED if not var4.get() else NORMAL)

            # # Desabilita os campos de entrada não associados ao checkbox marcado
            # input2.config(state=NORMAL if var2.get() else DISABLED)
            # input4.config(state=NORMAL if var4.get() else DISABLED)
            
        else:
            # Reabilita tudo se nenhum checkbox estiver marcado
            checkbox1.config(state=NORMAL)
            btn11.config(state=DISABLED)
            btn12.config(state=DISABLED)
            checkbox2.config(state=NORMAL)
            btn21.config(state=DISABLED)
            btn22.config(state=DISABLED)
            checkbox3.config(state=NORMAL)
            btn31.config(state=DISABLED)
            btn32.config(state=DISABLED)
            checkbox4.config(state=NORMAL)
            btn41.config(state=DISABLED)
            btn42.config(state=DISABLED)
            
        #     checkbox3.config(state=NORMAL)
        #     checkbox4.config(state=NORMAL)
        #     input2.config(state=DISABLED)
        #     input4.config(state=DISABLED)

        # # Limpa os inputs desativados
        # if not var2.get():
        #     input2.delete(0, "end")
        # if not var4.get():
        #     input4.delete(0, "end")


    # def_text = "\
    # Peso de alocação: 1\n\
    # Peso de troca de sala: 500\n\
    # Peso de agrupamento: 1\n\
    # Peso de superlotação: 10\n\
    # Índice de superlotação: 85%\n\
    # Quantidade de alunos da pós: 20"
    separator01 = ttk.Separator(frame1, orient="horizontal")
    separator01.grid(row=0, column=0, columnspan=2, sticky="ew", pady=(0, 10))
    # Checkbox 1
    texts = ttk.Separator(frame1, orient="horizontal")
    texts.grid(row=1, column=0, columnspan=2, sticky="", pady=(0,10))
    checkbox1 = Checkbutton(frame1, text="Modelo 1", variable=var1, command=atualizar_estado)
    checkbox1.grid(row=1, column=0, sticky="w", pady=5)
    lbl1 = tk.Label(frame1, text="Modelo sem índice de superlotação e sem agrupamento de cursos")
    lbl1.grid(row=2, column=0, sticky="w", pady=5)
    
    # Botões do Modelo 1 - só ficam habilitados se checkbox1 for selecionado
    btn11 = ttk.Button(frame1, text="Usar Configurações Padrão", state=DISABLED, 
                       command=lambda: roda_script(script="Modelo 1.py", 
                                                   nome=nome, 
                                                   peso_x="1",
                                                   peso_y="500",
                                                   peso_v="",
                                                   peso_z="",
                                                   alpha="",
                                                   qtd_pos="20"))
    # btn1.pack(pady=20)
    btn11.grid(row=3, column=0,pady=5, sticky="w")
    Tooltip(btn11, "\
    Peso de alocação: 1\n\
    Peso de troca de sala: 500\n\
    Peso de agrupamento: não usa\n\
    Peso de superlotação: não usa\n\
    Índice de superlotação: não usa\n\
    Quantidade de alunos da pós: 20")
    btn12 = ttk.Button(frame1, text="Usar Configurações Personalizadas", 
                       command=lambda: edit_config(file_name=nome, modelo='Modelo 1.py'), state=DISABLED)
    btn12.grid(row=3, column=1, pady=5, sticky="")

    separator11 = ttk.Separator(frame1, orient="horizontal")
    separator11.grid(row=4, column=0, columnspan=2, sticky="ew", pady=(0, 10))
    
    
    # Checkbox 2
    checkbox2 = Checkbutton(frame1, text="Modelo 2 - Recomendado", variable=var2, command=atualizar_estado)
    checkbox2.grid(row=5, column=0, sticky="w", pady=5)
    lbl2 = tk.Label(frame1, text="Modelo com índice de superlotação e sem agrupamento de cursos")
    lbl2.grid(row=6, column=0, sticky="w", pady=5)
    # Botões do Modelo 1 - só ficam habilitados se checkbox1 for selecionado
    btn21 = ttk.Button(frame1, text="Usar Configurações Padrão", state=DISABLED, 
                       command=lambda: roda_script(script="Modelo 2.py", 
                                                   nome=nome, 
                                                   peso_x="1",
                                                   peso_y="500",
                                                   peso_v="",
                                                   peso_z="10",
                                                   alpha="0.85",
                                                   qtd_pos="20"))
    # btn1.pack(pady=20)
    btn21.grid(row=7, column=0,pady=5, sticky="w")
    Tooltip(btn21, "\
    Peso de alocação: 1\n\
    Peso de troca de sala: 500\n\
    Peso de agrupamento: não usa\n\
    Peso de superlotação: 10\n\
    Índice de superlotação: 0.85\n\
    Quantidade de alunos da pós: 20")
    btn22 = ttk.Button(frame1, text="Usar Configurações Personalizadas", 
                       command=lambda: edit_config(file_name=nome, modelo='Modelo 2.py'), state=DISABLED)
    btn22.grid(row=7, column=1, pady=5, sticky="")
    
    separator2 = ttk.Separator(frame1, orient="horizontal")
    separator2.grid(row=8, column=0, columnspan=2, sticky="ew", pady=(0, 10))
    # input2 = Entry(frame1, state=DISABLED)  # Input associado ao Checkbox 2
    # input2.grid(row=3, column=1,pady=5)
    

    # # Checkbox 3
    checkbox3 = Checkbutton(frame1, text="Modelo 3", variable=var3, command=atualizar_estado)
    checkbox3.grid(row=9, column=0, sticky="w", pady=5)
    lbl3 = tk.Label(frame1, text="Modelo sem índice de superlotação e com agrupamento de cursos")
    lbl3.grid(row=10, column=0, sticky="w", pady=5)
    # Botões do Modelo 1 - só ficam habilitados se checkbox1 for selecionado
    btn31 = ttk.Button(frame1, text="Usar Configurações Padrão", state=DISABLED, 
                       command=lambda: roda_script(script="Modelo 3.py", 
                                                   nome=nome, 
                                                   peso_x="1",
                                                   peso_y="500",
                                                   peso_v="1",
                                                   peso_z="",
                                                   alpha="",
                                                   qtd_pos="20"))
    # btn1.pack(pady=20)
    btn31.grid(row=11, column=0,pady=5, sticky="w")
    Tooltip(btn31, "\
    Peso de alocação: 1\n\
    Peso de troca de sala: 500\n\
    Peso de agrupamento: 1\n\
    Peso de superlotação: não usa\n\
    Índice de superlotação: não usa\n\
    Quantidade de alunos da pós: 20")
    btn32 = ttk.Button(frame1, text="Usar Configurações Personalizadas", 
                       command=lambda: edit_config(file_name=nome, modelo='Modelo 3.py'), state=DISABLED)
    btn32.grid(row=11, column=1, pady=5, sticky="")
    
    separator3 = ttk.Separator(frame1, orient="horizontal")
    separator3.grid(row=12, column=0, columnspan=2, sticky="ew", pady=(0, 10))
    # checkbox3 = Checkbutton(frame1, text="Modelo sem índice de superlotação e com agrupamento de cursos", variable=var3, command=atualizar_estado)
    # checkbox3.grid(row=4, column=0, sticky="w", pady=5)
    # separator3 = ttk.Separator(frame1, orient="horizontal")
    # separator3.grid(row=5, column=0, sticky="ew", pady=(0, 10))
    
    # # Checkbox 4
    checkbox4 = Checkbutton(frame1, text="Modelo 4", variable=var4, command=atualizar_estado)
    checkbox4.grid(row=13, column=0, sticky="w", pady=5)
    lbl4 = tk.Label(frame1, text="Modelo com índice de superlotação e com agrupamento de cursos")
    lbl4.grid(row=14, column=0, sticky="w", pady=5)
    # Botões do Modelo 1 - só ficam habilitados se checkbox1 for selecionado
    btn41 = ttk.Button(frame1, text="Usar Configurações Padrão", state=DISABLED, 
                       command=lambda: roda_script(script="Modelo 4.py", 
                                                   nome=nome, 
                                                   peso_x="1",
                                                   peso_y="500",
                                                   peso_v="1",
                                                   peso_z="10",
                                                   alpha="0.85",
                                                   qtd_pos="20"))
    # btn1.pack(pady=20)
    btn41.grid(row=15, column=0,pady=5, sticky="w")
    Tooltip(btn41, "\
    Peso de alocação: 1\n\
    Peso de troca de sala: 500\n\
    Peso de agrupamento: 1\n\
    Peso de superlotação: 10\n\
    Índice de superlotação: 0.85\n\
    Quantidade de alunos da pós: 20")
    btn42 = ttk.Button(frame1, text="Usar Configurações Personalizadas", 
                       command=lambda: edit_config(file_name=nome, modelo='Modelo 4.py'), state=DISABLED)
    btn42.grid(row=15, column=1, pady=5, sticky="")
    
    separator4 = ttk.Separator(frame1, orient="horizontal")
    separator4.grid(row=16, column=0, columnspan=2, sticky="ew", pady=(0, 10))
    # checkbox4 = Checkbutton(frame1, text="Modelo com índice de superlotação e com agrupamento de cursos", variable=var4, command=atualizar_estado)
    # checkbox4.grid(row=6, column=0, sticky="w", pady=5)
    # input4 = Entry(frame1, state=DISABLED)  # Input associado ao Checkbox 2
    # input4.grid(row=6, column=1,pady=5)
    # separator4 = ttk.Separator(frame1, orient="horizontal")
    # separator4.grid(row=7, column=0, sticky="ew", pady=(0, 10))

    # # em = ttk.Button(frame, text="Executar Modelo", command=lambda: menu_modelo(file_name=arquivo_selecionado1))
    # # em.grid(row=5, column=0, pady=5)
    # executor = ttk.Button(frame1, text="Executar Modelo", 
    #                       command=lambda: model_condition(file_name=nome, 
    #                                                       input2=input2.get(), input4=input4.get(), 
    #                                                       checkbox1=var1.get(), checkbox2=var2.get(), 
    #                                                       checkbox3=var3.get(), checkbox4=var4.get()))
    # executor.grid(row=8, column=0, pady=5)

#### Novo menu

In [1595]:
# Variáveis associadas aos checkboxes
def teste_menu_modelo(nome):
    # nome = file_name.get()
    if nome == "Selecione uma base de dados":
        messagebox.showwarning("Aviso", "Por favor, selecione uma base de dados.")
        return
    menu = tk.Toplevel(root)
    menu.geometry("+550+250")
    menu.title("Selecionar Modelo")
    
    # Frame para o botão e o campo
    frame1 = tk.Frame(menu)
    frame1.pack(pady=10, padx=10)
    
    # name = file_name.get()
    var1 = IntVar()
    var2 = IntVar()
    var3 = IntVar()
    var4 = IntVar()
    var5 = IntVar()
    var6 = IntVar()
    var7 = IntVar()
    
    def atualizar_estado():
        # Se qualquer checkbox estiver marcado, desabilite os outros
        if var1.get() or var2.get() or var3.get() or var4.get():
            # Desabilita os checkboxes não selecionados
            checkbox1.config(state=DISABLED if not var1.get() else NORMAL)
            btn11.config(state=DISABLED if not var1.get() else NORMAL)
            btn12.config(state=DISABLED if not var1.get() else NORMAL)
            checkbox2.config(state=DISABLED if not var2.get() else NORMAL)
            btn21.config(state=DISABLED if not var2.get() else NORMAL)
            btn22.config(state=DISABLED if not var2.get() else NORMAL)
            checkbox3.config(state=DISABLED if not var3.get() else NORMAL)
            btn31.config(state=DISABLED if not var3.get() else NORMAL)
            btn32.config(state=DISABLED if not var3.get() else NORMAL)
            checkbox4.config(state=DISABLED if not var4.get() else NORMAL)
            btn41.config(state=DISABLED if not var4.get() else NORMAL)
            btn42.config(state=DISABLED if not var4.get() else NORMAL)
            # checkbox3.config(state=DISABLED if not var3.get() else NORMAL)
            # checkbox4.config(state=DISABLED if not var4.get() else NORMAL)

            # # Desabilita os campos de entrada não associados ao checkbox marcado
            # input2.config(state=NORMAL if var2.get() else DISABLED)
            # input4.config(state=NORMAL if var4.get() else DISABLED)
            
        else:
            # Reabilita tudo se nenhum checkbox estiver marcado
            checkbox1.config(state=NORMAL)
            btn11.config(state=DISABLED)
            btn12.config(state=DISABLED)
            checkbox2.config(state=NORMAL)
            btn21.config(state=DISABLED)
            btn22.config(state=DISABLED)
            checkbox3.config(state=NORMAL)
            btn31.config(state=DISABLED)
            btn32.config(state=DISABLED)
            checkbox4.config(state=NORMAL)
            btn41.config(state=DISABLED)
            btn42.config(state=DISABLED)
            
        #     checkbox3.config(state=NORMAL)
        #     checkbox4.config(state=NORMAL)
        #     input2.config(state=DISABLED)
        #     input4.config(state=DISABLED)

        # # Limpa os inputs desativados
        # if not var2.get():
        #     input2.delete(0, "end")
        # if not var4.get():
        #     input4.delete(0, "end")


    # def_text = "\
    # Peso de alocação: 1\n\
    # Peso de troca de sala: 500\n\
    # Peso de agrupamento: 1\n\
    # Peso de superlotação: 10\n\
    # Índice de superlotação: 85%\n\
    # Quantidade de alunos da pós: 20"
    separator01 = ttk.Separator(frame1, orient="horizontal")
    separator01.grid(row=0, column=0, columnspan=2, sticky="ew", pady=(0, 10))
    # Checkbox 1
    texts = ttk.Separator(frame1, orient="horizontal")
    texts.grid(row=1, column=0, columnspan=2, sticky="", pady=(0,10))
    checkbox1 = Checkbutton(frame1, text="Modelo 1", variable=var1, command=atualizar_estado)
    checkbox1.grid(row=1, column=0, sticky="w", pady=5)
    lbl1 = tk.Label(frame1, text="Modelo sem índice de superlotação e sem agrupamento de cursos")
    lbl1.grid(row=2, column=0, sticky="w", pady=5)
    
    # Botões do Modelo 1 - só ficam habilitados se checkbox1 for selecionado
    btn11 = ttk.Button(frame1, text="Usar Configurações Padrão", state=DISABLED, 
                       command=lambda: roda_script(script="Modelo 1.py", 
                                                   nome=nome, 
                                                   peso_x="1",
                                                   peso_y="500",
                                                   peso_v="",
                                                   peso_z="",
                                                   alpha="",
                                                   qtd_pos="20"))
    # btn1.pack(pady=20)
    btn11.grid(row=3, column=0,pady=5, sticky="w")
    Tooltip(btn11, "\
    Peso de alocação: 1\n\
    Peso de troca de sala: 500\n\
    Peso de agrupamento: não usa\n\
    Peso de superlotação: não usa\n\
    Índice de superlotação: não usa\n\
    Quantidade de alunos da pós: 20")
    btn12 = ttk.Button(frame1, text="Usar Configurações Personalizadas", 
                       command=lambda: edit_config(file_name=nome, modelo='Modelo 1.py'), state=DISABLED)
    btn12.grid(row=3, column=1, pady=5, sticky="")

    separator11 = ttk.Separator(frame1, orient="horizontal")
    separator11.grid(row=4, column=0, columnspan=2, sticky="ew", pady=(0, 10))
    
    
    # Checkbox 2
    checkbox2 = Checkbutton(frame1, text="Modelo 2 - Recomendado", variable=var2, command=atualizar_estado)
    checkbox2.grid(row=5, column=0, sticky="w", pady=5)
    lbl2 = tk.Label(frame1, text="Modelo com índice de superlotação e sem agrupamento de cursos")
    lbl2.grid(row=6, column=0, sticky="w", pady=5)
    # Botões do Modelo 1 - só ficam habilitados se checkbox1 for selecionado
    btn21 = ttk.Button(frame1, text="Usar Configurações Padrão", state=DISABLED, 
                       command=lambda: roda_script(script="Modelo 2.py", 
                                                   nome=nome, 
                                                   peso_x="1",
                                                   peso_y="500",
                                                   peso_v="",
                                                   peso_z="10",
                                                   alpha="0.85",
                                                   qtd_pos="20"))
    # btn1.pack(pady=20)
    btn21.grid(row=7, column=0,pady=5, sticky="w")
    Tooltip(btn21, "\
    Peso de alocação: 1\n\
    Peso de troca de sala: 500\n\
    Peso de agrupamento: não usa\n\
    Peso de superlotação: 10\n\
    Índice de superlotação: 0.85\n\
    Quantidade de alunos da pós: 20")
    btn22 = ttk.Button(frame1, text="Usar Configurações Personalizadas", 
                       command=lambda: edit_config(file_name=nome, modelo='Modelo 2.py'), state=DISABLED)
    btn22.grid(row=7, column=1, pady=5, sticky="")
    
    separator2 = ttk.Separator(frame1, orient="horizontal")
    separator2.grid(row=8, column=0, columnspan=2, sticky="ew", pady=(0, 10))
    # input2 = Entry(frame1, state=DISABLED)  # Input associado ao Checkbox 2
    # input2.grid(row=3, column=1,pady=5)
    

    # # Checkbox 3
    checkbox3 = Checkbutton(frame1, text="Modelo 3", variable=var3, command=atualizar_estado)
    checkbox3.grid(row=9, column=0, sticky="w", pady=5)
    lbl3 = tk.Label(frame1, text="Modelo sem índice de superlotação e com agrupamento de cursos")
    lbl3.grid(row=10, column=0, sticky="w", pady=5)
    # Botões do Modelo 1 - só ficam habilitados se checkbox1 for selecionado
    btn31 = ttk.Button(frame1, text="Usar Configurações Padrão", state=DISABLED, 
                       command=lambda: roda_script(script="Modelo 3.py", 
                                                   nome=nome, 
                                                   peso_x="1",
                                                   peso_y="500",
                                                   peso_v="1",
                                                   peso_z="",
                                                   alpha="",
                                                   qtd_pos="20"))
    # btn1.pack(pady=20)
    btn31.grid(row=11, column=0,pady=5, sticky="w")
    Tooltip(btn31, "\
    Peso de alocação: 1\n\
    Peso de troca de sala: 500\n\
    Peso de agrupamento: 1\n\
    Peso de superlotação: não usa\n\
    Índice de superlotação: não usa\n\
    Quantidade de alunos da pós: 20")
    btn32 = ttk.Button(frame1, text="Usar Configurações Personalizadas", 
                       command=lambda: edit_config(file_name=nome, modelo='Modelo 3.py'), state=DISABLED)
    btn32.grid(row=11, column=1, pady=5, sticky="")
    
    separator3 = ttk.Separator(frame1, orient="horizontal")
    separator3.grid(row=12, column=0, columnspan=2, sticky="ew", pady=(0, 10))
    # checkbox3 = Checkbutton(frame1, text="Modelo sem índice de superlotação e com agrupamento de cursos", variable=var3, command=atualizar_estado)
    # checkbox3.grid(row=4, column=0, sticky="w", pady=5)
    # separator3 = ttk.Separator(frame1, orient="horizontal")
    # separator3.grid(row=5, column=0, sticky="ew", pady=(0, 10))
    
    # # Checkbox 4
    checkbox4 = Checkbutton(frame1, text="Modelo 4", variable=var4, command=atualizar_estado)
    checkbox4.grid(row=13, column=0, sticky="w", pady=5)
    lbl4 = tk.Label(frame1, text="Modelo com índice de superlotação e com agrupamento de cursos")
    lbl4.grid(row=14, column=0, sticky="w", pady=5)
    # Botões do Modelo 1 - só ficam habilitados se checkbox1 for selecionado
    btn41 = ttk.Button(frame1, text="Usar Configurações Padrão", state=DISABLED, 
                       command=lambda: roda_script(script="Modelo 4.py", 
                                                   nome=nome, 
                                                   peso_x="1",
                                                   peso_y="500",
                                                   peso_v="1",
                                                   peso_z="10",
                                                   alpha="0.85",
                                                   qtd_pos="20"))
    # btn1.pack(pady=20)
    btn41.grid(row=15, column=0,pady=5, sticky="w")
    Tooltip(btn41, "\
    Peso de alocação: 1\n\
    Peso de troca de sala: 500\n\
    Peso de agrupamento: 1\n\
    Peso de superlotação: 10\n\
    Índice de superlotação: 0.85\n\
    Quantidade de alunos da pós: 20")
    btn42 = ttk.Button(frame1, text="Usar Configurações Personalizadas", 
                       command=lambda: edit_config(file_name=nome, modelo='Modelo 4.py'), state=DISABLED)
    btn42.grid(row=15, column=1, pady=5, sticky="")
    
    separator4 = ttk.Separator(frame1, orient="horizontal")
    separator4.grid(row=16, column=0, columnspan=2, sticky="ew", pady=(0, 10))
    # checkbox4 = Checkbutton(frame1, text="Modelo com índice de superlotação e com agrupamento de cursos", variable=var4, command=atualizar_estado)
    # checkbox4.grid(row=6, column=0, sticky="w", pady=5)
    # input4 = Entry(frame1, state=DISABLED)  # Input associado ao Checkbox 2
    # input4.grid(row=6, column=1,pady=5)
    # separator4 = ttk.Separator(frame1, orient="horizontal")
    # separator4.grid(row=7, column=0, sticky="ew", pady=(0, 10))

    # # em = ttk.Button(frame, text="Executar Modelo", command=lambda: menu_modelo(file_name=arquivo_selecionado1))
    # # em.grid(row=5, column=0, pady=5)
    # executor = ttk.Button(frame1, text="Executar Modelo", 
    #                       command=lambda: model_condition(file_name=nome, 
    #                                                       input2=input2.get(), input4=input4.get(), 
    #                                                       checkbox1=var1.get(), checkbox2=var2.get(), 
    #                                                       checkbox3=var3.get(), checkbox4=var4.get()))
    # executor.grid(row=8, column=0, pady=5)

#### Editar Configs (obsoleto)

In [1596]:
def edit_config(file_name, modelo):
    if file_name == "Selecione uma base de dados":
        messagebox.showwarning("Aviso", "Por favor, insira o nome da nova base de dados.")
        return
        
    configs = tk.Toplevel(root)
    configs.geometry("+750+450")
    configs.title("Personalizar parâmetros")
    
    # Frame para o botão e o campo
    frame2 = tk.Frame(configs)
    frame2.pack(pady=10, padx=10)
    
    ttk.Separator(frame2, orient="horizontal").grid(row=0, column=0, columnspan=2, sticky="ew", pady=(0, 10))
    lbl = tk.Label(frame2, text="Digite os valores que deseja para os parâmetros.", state=DISABLED)
    lbl.grid(row=1,column=0,sticky='w')
    btnl = ttk.Button(frame2, text="Recomendações - Configurações Padrão", state=DISABLED)
    btnl.grid(row=1,column=1,sticky='e')
    Tooltip(btnl, "Recomendado:\n\
    Peso de alocação: 1\n\
    Peso de troca de sala: 500\n\
    Peso de agrupamento: 1\n\
    Peso de superlotação: 10\n\
    Índice de superlotação: 0.85\n\
    Quantidade de alunos da pós: 20")

    ttk.Separator(frame2, orient="horizontal").grid(row=2, column=0, columnspan=2, sticky="ew", pady=(0, 10))
    lblx = tk.Label(frame2, text="Defina um peso de alocação:")
    lblx.grid(row=3, column=0, sticky='w', pady=5)
    peso_x = Entry(frame2, state=DISABLED)
    peso_x.grid(row=3, column=2, sticky='e', pady=5)
    Tooltip(peso_x, "O peso de alocação refere-se ao peso da variável x_as no modelo. \n\
    Quanto mais alto, mais importante será essa variável,\n\
    logo, o modelo pode deixar de alocar de forma ótima se o peso for baixo.\n\
    Por exemplo, ele pode entender que é mais importante agrupar \n\
    os cursos, do que usar o maior espaço possível.")
    
    ttk.Separator(frame2, orient="horizontal").grid(row=4, column=0, columnspan=2, sticky="ew", pady=(0, 10))
    lbly = tk.Label(frame2, text="Defina um peso de troca de sala:")
    lbly.grid(row=5,column=0,sticky='w',pady=5)
    peso_y = Entry(frame2, state=DISABLED)
    peso_y.grid(row=5,column=2,sticky='e',pady=5)
    Tooltip(peso_y, "O peso de troca de sala refere-se ao peso da variável y_t no modelo. \n\
    Quanto mais alto, mais importante será essa variável,\n\
    logo, o modelo pode deixar de manter aulas de uma \n\
    turma/disciplina alocadas na mesma sala se o peso for baixo.\n\
    Por exemplo, ele pode entender que é mais importante preencher \n\
    uma sala o máximo possível sem verificar quais são elas.")
    
    ttk.Separator(frame2, orient="horizontal").grid(row=6, column=0, columnspan=2, sticky="ew", pady=(0, 10))
    lblv = tk.Label(frame2, text="Defina um peso de agrupamento:")
    lblv.grid(row=7,column=0,sticky='w',pady=5)
    peso_v = Entry(frame2, state=DISABLED)
    peso_v.grid(row=7,column=2,sticky='e',pady=5)
    Tooltip(peso_v, "O peso de agrupamento refere-se ao peso da variável v_cssl no modelo. \n\
    Quanto mais alto, mais importante será essa variável,\n\
    logo, o modelo pode deixar de tentar agrupar as aulas de \n\
    um curso próximas umas das outras se o peso for baixo.\n\
    Por exemplo, ele pode entender que é mais importante \n\
    acomodar os alunos numa sala, do que agrupar os cursos.")

    ttk.Separator(frame2, orient="horizontal").grid(row=8, column=0, columnspan=2, sticky="ew", pady=(0, 10))
    lblz = tk.Label(frame2, text="Defina um peso de superlotação:")
    lblz.grid(row=9,column=0,sticky='w',pady=5)
    peso_z = Entry(frame2, state=DISABLED)
    peso_z.grid(row=9,column=2,sticky='e',pady=5)
    Tooltip(peso_z, "O peso de superlotação refere-se ao peso da variável z_as no modelo. \n\
    Quanto mais alto, mais importante será essa variável,\n\
    logo, o modelo pode deixar de tentar acomodar melhor os \n\
    alunos em uma sala se o peso for baixo.\n\
    Por exemplo, ele pode entender que é mais importante \n\
    preencher uma sala o máximo possível, do que deixar alguns lugares desocupados \n\
    para acomodar melhor os alunos, ou para receber alunos vindos do requerimento.")

    ttk.Separator(frame2, orient="horizontal").grid(row=10, column=0, columnspan=2, sticky="ew", pady=(0, 10))
    lbla = tk.Label(frame2, text="Defina um índice de superlotação (valor entre 0 e 1, utilizando '.'):")
    lbla.grid(row=11,column=0,sticky='w',pady=5)
    alpha = Entry(frame2, state=DISABLED)
    alpha.grid(row=11,column=2,sticky='e',pady=5)
    Tooltip(alpha, "O índice de superlotação refere-se à porcentagem usada pela variável z_as no modelo. \n\
    Em outras palavras, é o quanto uma aula preenche uma sala \n\
    para que o modelo tente colocar essa aula em uma sala maior, \n\
    garantindo uma melhor acomodação para os alunos e espaço para \n\
    alunos vindos do requerimento. Quanto mais alto, mais o modelo \n\
    prioriza preencher o máximo possível de uma sala. Um valor baixo \n\
    faz com que as salas maiores sejam preenchidas com mais facilidade, \n\
    e um valor alto fará o mesmo, mas para as salas menores.")

    ttk.Separator(frame2, orient="horizontal").grid(row=12, column=0, columnspan=2, sticky="ew", pady=(0, 10))
    lblq = tk.Label(frame2, text="Defina uma quantidade de alunos da pós para as disciplinas com espelho:")
    lblq.grid(row=13,column=0,sticky='w',pady=5)
    qtd_pos = Entry(frame2, state=DISABLED)
    qtd_pos.grid(row=13,column=2,sticky='e',pady=5)
    Tooltip(qtd_pos, "A quantidade de alunos da pós refere-se a um valor sugerido pelo usuário \n\
    para simbolizar quantos alunos da pós-graduação farão \n\
    uma disciplina da graduação que é espelhada com a pós. \n\
    Esse valor será aplicado no número de inscritos de QUALQUER \n\
    disciplina que tenha espelho com a pós.")

    
    if modelo == 'Modelo 1.py':
        peso_x.config(state=NORMAL)
        peso_y.config(state=NORMAL)
        qtd_pos.config(state=NORMAL)
    elif modelo == 'Modelo 2.py':
        peso_x.config(state=NORMAL)
        peso_y.config(state=NORMAL)
        peso_z.config(state=NORMAL)
        alpha.config(state=NORMAL)
        qtd_pos.config(state=NORMAL)
    elif modelo == 'Modelo 3.py':
        peso_x.config(state=NORMAL)
        peso_y.config(state=NORMAL)
        peso_v.config(state=NORMAL)
        qtd_pos.config(state=NORMAL)
    elif modelo == 'Modelo 4.py':
        peso_x.config(state=NORMAL)
        peso_y.config(state=NORMAL)
        peso_v.config(state=NORMAL)
        peso_z.config(state=NORMAL)
        alpha.config(state=NORMAL)
        qtd_pos.config(state=NORMAL)
    elif modelo == 'verificar_dados.py':
        qtd_pos.config(state=NORMAL)
    else:
        messagebox.showwarning("Aviso", "Um erro inesperado ocorreu.")
        return
        # roda_script(modelo, file_name)
    ttk.Separator(frame2, orient="horizontal").grid(row=14, column=0, columnspan=2, sticky="ew", pady=(0, 10))
    ttk.Button(frame2, text="Executar", 
               command=lambda: execute_configs(nome=file_name, modelo=modelo, peso_x=peso_x, peso_y=peso_y, 
                                               peso_v=peso_v, peso_z=peso_z, alpha=alpha, qtd_pos=qtd_pos)).grid(row=15, column=0, sticky='w', pady=5)
    
    

In [1597]:
def execute_configs(nome, modelo, peso_x, peso_y, peso_v, peso_z, alpha, qtd_pos):
    if not peso_x.get():
        if modelo == 'Modelo 1.py' or modelo == 'Modelo 2.py' or modelo == 'Modelo 3.py' or modelo == 'Modelo 4.py':
            messagebox.showwarning("Aviso", "Preencha o campo do 'Peso de alocação'.")
            return
    if not peso_y.get():
        if modelo == 'Modelo 1.py' or modelo == 'Modelo 2.py' or modelo == 'Modelo 3.py' or modelo == 'Modelo 4.py':
            messagebox.showwarning("Aviso", "Preencha o campo do 'Peso de troca de sala'.")
            return
    if not peso_v.get():
        if modelo == 'Modelo 3.py' or modelo == 'Modelo 4.py':
            messagebox.showwarning("Aviso", "Preencha o campo do 'Peso de agrupamento'.")
            return
    if not peso_z.get():
        if modelo == 'Modelo 2.py' or modelo == 'Modelo 4.py':
            messagebox.showwarning("Aviso", "Preencha o campo do 'Peso de superlotação'.")
            return
    if not alpha.get():
        if modelo == 'Modelo 2.py' or modelo == 'Modelo 4.py':
            messagebox.showwarning("Aviso", "Preencha o campo do 'Índice de superlotação'.")
            return
    if not qtd_pos.get():
        if modelo == 'Modelo 1.py' or modelo == 'Modelo 2.py' or modelo == 'Modelo 3.py' or modelo == 'Modelo 4.py' or modelo == 'verificar_dados.py':
            messagebox.showwarning("Aviso", "Preencha o campo de 'Quantidade de alunos da pós'.")
            return

    roda_script(modelo, nome, peso_x.get(), peso_y.get(), peso_v.get(), peso_z.get(), alpha.get(), qtd_pos.get())
    # teste()

In [1598]:
def teste():
    messagebox.showinfo("Sucesso", "Yipee")

def teste2():
    messagebox.showinfo("Success", "S U C C")

## Análise de Espaços Livres (colocar nos arquivos de modelo)

In [1599]:
# Função que cria uma janela para selecionar arquivos de solução do modelo, que serão analisados 
# para criar planilhas mostrando os espaços livres após as alocações.
def analise_vazios():
    # Crio uma nova janela em cima da janela principal da interface.
    nova_janela = tk.Toplevel(root)
    nova_janela.title("Análise de Espaços Livres")
    
    # Crio o frame para armazenar os botões e outros campos da nova janela.
    frame = tk.Frame(nova_janela)
    frame.pack(pady=10, padx=10)

    # Defino uma variável para salvar o arquivo com a planilha de Visualização Completa da Solução do modelo.
    visu = tk.StringVar(value="Selecione a planilha de Visualização Completa da Solução do modelo")

    # Defino uma variável para salvar o arquivo com a planilha dos dados das salas.
    salas = tk.StringVar(value="Selecione a planilha dos dados das salas")
    
    # Defino uma variável para salvar o nome, fornecido pelo usuário, para o arquivo de Visualização de Espaços Livres.
    caminho_arquivo = tk.StringVar()

    # Defino uma variável para salvar o nome, fornecido pelo usuário, para o arquivo de Planilha de Espaços Livres.
    caminho_arquivo1 = tk.StringVar()

    # Defino as funções para selecionar arquivos.
    def selecionar_arquivo1():
        # O usuário seleciona o arquivo contendo a base de dados das aulas.
        arquivo = filedialog.askopenfilename(title="Selecione a planilha de Visualização Completa da Solução do modelo")

        # Se um arquivo foi selecionado:
        if arquivo:
            # Salvo o caminho do arquivo.
            visu.set(arquivo)

    # Função análoga.
    def selecionar_arquivo2():
        # O usuário seleciona o arquivo contendo a base de dados das aulas.
        arquivo = filedialog.askopenfilename(title="Selecione a planilha dos dados das salas")

        # Se um arquivo foi selecionado:
        if arquivo:
            # Salvo o caminho do arquivo.
            salas.set(arquivo)

    # Defino uma função que salva os valores das variáveis contendo o nome dos arquivos escolhidos.
    def salvar_valores():
        # Todas as condições a seguir seguem a lógica de que, se um arquivo não foi selecionado, uma janela avisando o ocorrido
        # aparece, pedindo para o usuário selecionar um arquivo no campo requerido, ou fornecer um nome para os arquivos
        # que serão criados.
        
        if not visu.get() or visu.get() == "Selecione a planilha de Visualização Completa da Solução do modelo":
            messagebox.showwarning("Aviso", "Por favor, selecione a planilha de Visualização Completa da Solução do modelo.")
            return
        if not salas.get() or salas.get() == "Selecione a planilha dos dados das salas":
            messagebox.showwarning("Aviso", "Por favor, selecione a planilha dos dados das salas.")
            return
        if not caminho_arquivo.get():
            messagebox.showwarning("Aviso", "Por favor, dê um nome para o arquivo de Visualização de Espaços Livres.")
            return
        if not caminho_arquivo1.get():
            messagebox.showwarning("Aviso", "Por favor, dê um nome para o arquivo de Planilha de Espaços Livres.")
            return
        

        # Com o arquivo necessário selecionado, e os nomes dos novos arquivos obtidos, chamo a função que fará a análise dos espaços livres.
        criar_visualizacao_vazia(visu.get(), salas.get(), caminho_arquivo.get(), caminho_arquivo1.get(), )

        # E destruo a janela após a conclusão do processo.
        nova_janela.destroy()

    # Crio uma legenda para ficar ao lado do botão.
    lbl_visu = tk.Label(frame, text="Selecione a de Dados da Solução do modelo")
    # Defino a posição do texto na janela.
    lbl_visu.grid(row=0, column=0, pady=5, sticky="w")
    # Crio o botão para salvar o arquivo da Visualização Completa da Solução do modelo.
    btn_selecionar_visu = tk.Button(frame, textvariable=visu, command=selecionar_arquivo1, wraplength=250, width=40)
    # Defino a posição do botão na janela.
    btn_selecionar_visu.grid(row=0, column=1, padx=5, pady=5)

    # As linhas a seguir são análogas.
    lbl_salas = tk.Label(frame, text="Selecione a planilha dos dados das salas")
    lbl_salas.grid(row=1, column=0, pady=5, sticky="w")
    btn_selecionar_salas = tk.Button(frame, textvariable=salas, command=selecionar_arquivo2, wraplength=250, width=40)
    btn_selecionar_salas.grid(row=1, column=1, padx=5, pady=5)
    
    lbl_cam = tk.Label(frame, text="Digite um nome para a Visualização de Espaços Livres:")
    lbl_cam.grid(row=2, column=0, pady=5, sticky="w")
    campo_cam = tk.Entry(frame, textvariable=caminho_arquivo)
    campo_cam.grid(row=2, column=1, pady=5)

    lbl_cam1 = tk.Label(frame, text="Digite um nome para a Planilha de Espaços Livres:")
    lbl_cam1.grid(row=3, column=0, pady=5, sticky="w")
    campo_cam1 = tk.Entry(frame, textvariable=caminho_arquivo1)
    campo_cam1.grid(row=3, column=1, pady=5)

    # Defino um botção e sua posição na janela, cujo papel é chamar a função que salva os valores dos arquivos para a análise.
    ttk.Button(frame, text="Gerar Análise de Espaços Livres", command=salvar_valores).grid(row=4,column=0,sticky='ew')

### Criar Visualização em cima de Visualização

In [ ]:
# Função que faz a análise de espaços livres restantes após a alocação feita pelo modelo.
def criar_visualizacao_vazia(file_path, file_path_salas, caminho_arquivo, caminho_arquivo1):
    # Defino uma variável que recebe a planilha do Excel da Visualização Completa da Solução do modelo.
    wb = load_workbook(file_path)
    ws = wb.active

    # Defino duas variáveis com o horário inicial e final, isto é, o intervalo dos horários que serão colocados na planilha para a visualização.
    start_time = datetime.strptime("07:00", "%H:%M")
    end_time = datetime.strptime("23:30", "%H:%M")
    
    # Crio uma lista para conter os valores do intervalo de horários.
    horarios = []
    
    # Crio uma variável com o horário inicial.
    current_time = start_time
    
    # Enquanto o horário inicial não for maior que o horário final, isto é, enquanto houver valores para serem colocados no intervalo:
    while current_time <= end_time:
        # Adiciono o horário atual na lista de horários.
        horarios.append(current_time.strftime("%H:%M"))
        
        # Faço um acréscimo de 30 minutos no horário atual.
        current_time += timedelta(minutes=30)
    
    # Crio uma variável com a quantidade de colunas necessárias para colocar todos os horários do intervalo.
    sala_colunas = len(horarios)
    
    # Lista com os nomes dos dias da semana que serão usados na visualização.
    dias_semana = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta", "Sábado"]
    
    # Defino as linhas e colunas de início e fim para a primeira tabela da planilha, isto é, para preencher os dados da primeira sala.
    start_row = 1 # Linha de início.
    start_column = 2 # Coluna de início.
    end_row = 1 # Linha de término.
    end_column = 1 + sala_colunas # Coluna de término.
    start = start_row + 2 # Linha de início para listar os dias da semana.
    space_between = 3 # Número de linhas entre a tabela de uma sala para a de outra sala.

    # Defino um preenchimento amarelo para simbolizar os espaços livres das salas.
    yellow_fill = PatternFill(start_color="00E3DE00", end_color="00E3DE00", fill_type="solid")

    # Defino uma variável que guarda o número de salas utilizadas na solução do modelo.
    # O cálculo feito é baseado no comprimento da planilha, no número de dias da semana mostrados na visualização, e no
    # espaço entre cada "tabela" de cada sala.
    n_salas = int((ws.max_row+1) / (2 + len(dias_semana) + space_between))

    # Salvo os dados das salas em uma variável.
    salas = pd.read_excel(file_path_salas)

    # Crio uma lista para salvar quais salas foram usadas para a alocação do modelo.
    salas1 = []

    # Crio um dataframe vazio para salvar os intervalos de tempo onde as salas estão disponíveis.
    df_vazio = pd.DataFrame(columns=["Sala", "Dia da semana", "Horário vago"])
    
    # Função para obter a cor da célula.
    def get_cell_color(cell):
        # Salvo a cor de preenchimento da célula em uma variável.
        fill = cell.fill
        
        # if fill and fill.start_color.index != "ffffff":  # Ignorar células sem cor
        # Verifico se tinha algum preenchimento diferente, ou se o preenchimento é amarelo:
        if fill and fill.start_color.index != "00E3DE00":  # Ignorar células amarelas
            # Se houver, retorno o código do preenchimento.
            return fill.start_color.index  # Retorna o código da cor

        # Caso não houver uma cor de preenchimento definida, retorno vazio.
        return None  # Sem cor definida

    # Para cada sala utilizada pelo modelo:
    for i in range(n_salas+1):
        # print(type(ws.cell(row=start_row, column=start_column)))
        # Salvo a sala que está sendo analisada atualmente em uma variável.
        sala = ws.cell(row=start_row, column=start_column).value

        # E também a adiciono na lista de salas usadas.
        salas1.append(sala)
        
        # Para cada linha da planilha:
        for row in ws.iter_rows(min_row=start, max_row=start+5):
        # for i, dia in enumerate(dias_semana, start=start):

            # Defino a coluna inicial como vazia.
            start_col = None  # Início da mesclagem
            # current_color = None  # Cor atual

            # Para cada par (coluna, célula) da linha sendo analisada:
            for col_idx, cell in enumerate(row, start=1):
                # Busco identificar a cor do preenchimento da célula.
                cell_color = get_cell_color(cell)

                # Se a cor da célula atual for branca, possivelmente deixei de estar analisando um horário ocupado:
                if cell_color == "00000000":
                    # Verifico se a célula é uma célula mesclada:
                    if type(cell) != openpyxl.cell.cell.MergedCell:
                        # Se não for, verifico se a coluna inicial está como vazia,
                        # e se a célula atual não faz parte da coluna de dias da semana:
                        if start_col is None and cell.value not in dias_semana:
                            # Se for o caso, salvo a coluna inicial na variável.
                            # O workbook sempre considera a primeira coluna como 1, e como o primeiro "elemento" da lista de horários está na
                            # coluna 2, então para traduzir corretamente a coluna da célula vazia, preciso tirar 2 para o índice ficar compatível
                            start_col = cell.column - 2

                # Se a cor da célula atual não for branca:
                else:
                    # Verifico se a célula que estou analisando não é mesclada:
                    if type(cell) != openpyxl.cell.cell.MergedCell:
                        # Se ela não for, quer dizer que estou analisando a primeira célula de um intervalo ocupado, isto é,
                        # estou analisando a célula do topo esquerdo da mesclagem, que é a única que pode ter seu valor alterado.
                        # Assim, limpo o que estiver dentro da célula.
                        cell.value = ""

                        # E mudo a cor de seu preenchimento para o amarelo definido anteriormente.
                        cell.fill = yellow_fill
                        
                    # Agora, verifico se a coluna inicial é diferente de vazio, ou seja, se ela já foi definida;
                    # e verifico se a distância entre a coluna atual e a coluna inicial é maior ou igual que 1,
                    # certificando que não estou olhando para a mesma coluna:
                    if start_col is not None and col_idx - start_col >= 1:
                        # Aqui é onde eu vou pegar os intervalos do merge
                        # print(f"a: {start_col}: {horarios[start_col-1]} - {horarios[col_idx-1-2+1]}\n")
                        
                        # Se isso for verdade, adiciono uma linha no dataframe contendo os horários disponíveis até então.
                        # Na ordem, esta a sala que estou analisando, o dia da semana com o horário disponível, e
                        # o intervalo de tempo livre do dia em questão. 
                        # Como estou analisando linha por linha, a coluna do segundo valor sempre é 1, pois refere-se 
                        # à coluna de dias da semana; e eu preciso subtrair o valor 2 de col_idx para ignorar a primeira coluna
                        # da tabela (-1), e para igualar com o índice usual (já que col_idx sempre começa com o valor 1).
                        df_vazio.loc[len(df_vazio)] = [f"{sala}", f"{ws.cell(row=cell.row, column=1).value}", 
                                                       f"{horarios[start_col]} - {horarios[col_idx-2]}"]

                    # Após estes processos, defino novamente a coluna de início como vazia para definir um novo intervalo.
                    start_col = None
                    
        
            # Por garantia, adiciono o último intervalo de horário disponível nos dados, já que a análise da linha
            # poderia terminar e ir para a próxima sem incluir o último horário disponível dela:
            if start_col is not None and start_col < len(row):
                # print(f"b: {start_col}: {horarios[start_col-1]} - {horarios[col_idx-1-2+1]}\n")
                # Incluo esses dados da mesma forma como anteriormente.
                df_vazio.loc[len(df_vazio)] = [f"{sala}", f"{ws.cell(row=cell.row, column=1).value}", 
                                               f"{horarios[start_col]} - {horarios[col_idx-2]}"]
        
        # Feito isso, atualizo meus dados de criação, isto é, as coordenadas de onde a tabela da sala seguinte será colocada na planilha.
        start_row = start_row + 2 + len(dias_semana) + space_between
        end_row = start_row
        start = start_row + 2        


    # Após fazer a análise de espaços livres de todas as salas utilizadas na alocação, verificarei quais ficaram inteiramente de fora.
    # Para isso, verifico cada sala na planilha de salas:
    for sala in salas['Sala'].tolist():
        # Se a sala não está na lista de salas utilizadas:
        if sala not in salas1:
            
            if sala == '6-303' and '6-303/6-304' in salas1:
                continue
            elif sala == '6-304' and '6-303/6-304' in salas1:
                continue
            elif sala == '6-305' and '6-305/6-306' in salas1:
                continue
            elif sala == '6-306' and '6-305/6-306' in salas1:
                continue
            else:
                # Adiciono qual sala não foi utilizada na planilha de dados de espaços livres.
                # As condições dadas anteriormente são para garantir que o código não inclua as salas de laboratório avulsas
                # quando apenas as em conjunto foram utilizadas. Por exemplo, se o modelo tiver alocado apenas os laboratórios
                # do bloco 6 (6-303/6-304 e 6-305/6-306), o algoritmo pode incluir, erroneamente, que a sala 6-303 não foi usada.
                df_vazio.loc[len(df_vazio)+1, 'Sala'] = f"A sala {sala} não foi usada pela graduação."
                
    # caminho_arquivo = f"Visualização de Espaços Livres - Teste"
    # caminho_arquivo1 = f"Planilha com Espaços Livres - Teste"

    # Como visto anteriormente, esse trecho do código serve para salvar o arquivo sem sobrescrever um outro, alterando
    # o nome fornecido pelo usuário com um valor de "cópia".
    full_name = caminho_arquivo + ".xlsx"
    # i = 1
    # while os.path.exists(full_name):
    #     full_name = f"{caminho_arquivo} ({i}){'.xlsx'}"
    #     i += 1
    caminho_arquivo = full_name

    full_name = caminho_arquivo1 + ".xlsx"
    # i = 1
    # while os.path.exists(full_name):
    #     full_name = f"{caminho_arquivo1} ({i}){'.xlsx'}"
    #     i += 1

    caminho_arquivo1 = full_name

    # Por fim, salvo os arquivos criados com seus respectivos métodos.
    wb.save(caminho_arquivo)
    df_vazio.to_excel(caminho_arquivo1, sheet_name="Resultados", index=False)

    # E abro uma janela avisando o usuário de que os arquivos foram salvos.
    messagebox.showinfo("", f"Arquivo {caminho_arquivo} criado com sucesso!\n\nArquivo {caminho_arquivo1} criado com sucesso!")




# Preencher Planilha de Dados

In [1601]:
# Função que preenche as planilhas de dados fornecidas pelo usuário, sem a necessidade do mesmo preencher as
# alocações feitas pelo modelo de forma manual.
# Estruturalmente, a função é bem semelhante à de construir a primeira base de dados (a das aulas), com a única diferença 
# sendo dois campos a mais para inserir a bases de dados feita anteriormente, e a solução dada pelo modelo.
def preencher_planilha_dados():

    # Crio uma nova janela em cima da janela principal da interface.
    nova_janela = tk.Toplevel(root)
    nova_janela.title("Selecionar Arquivo e Inserir Valor")

    # Crio o frame para armazenar os botões e outros campos da nova janela.
    frame = tk.Frame(nova_janela)
    frame.pack(pady=10, padx=10)

    # Defino várias variáveis para armazenar os nomes dos arquivos que serão preenchidos.
    arquivo_sme = tk.StringVar(value="Selecione a planilha do SME")
    arquivo_sma = tk.StringVar(value="Selecione a planilha do SMA")
    arquivo_scc = tk.StringVar(value="Selecione a planilha do SCC")
    arquivo_ssc = tk.StringVar(value="Selecione a planilha do SSC")
    arquivo_outros = tk.StringVar(value="Selecione a planilha dos Outros Institutos")
    arquivo_sol = tk.StringVar(value="Selecione a planilha com os Dados da Solução do Modelo")
    arquivo_base = tk.StringVar(value="Selecione a Base de Dados")
    
    

    # Defino funções para selecionar arquivos.
    def selecionar_sme():
        # O usuário seleciona o arquivo contendo a base de dados das aulas.
        arquivo = filedialog.askopenfilename(title="Selecione a planilha do SME")

        # Se um arquivo foi selecionado:
        if arquivo:
            # Salvo o caminho do arquivo.
            arquivo_sme.set(arquivo)
    def selecionar_sma():
        arquivo = filedialog.askopenfilename(title="Selecione a planilha do SMA")
        if arquivo:
            arquivo_sma.set(arquivo)
    def selecionar_scc():
        arquivo = filedialog.askopenfilename(title="Selecione a planilha do SCC")
        if arquivo:
            arquivo_scc.set(arquivo)
    def selecionar_ssc():
        arquivo = filedialog.askopenfilename(title="Selecione a planilha do SSC")
        if arquivo:
            arquivo_ssc.set(arquivo)
    def selecionar_outros():
        arquivo = filedialog.askopenfilename(title="Selecione a planilha dos Outros Institutos")
        if arquivo:
            arquivo_outros.set(arquivo)
    def selecionar_sol():
        arquivo = filedialog.askopenfilename(title="Selecione a planilha com os Dados da Solução do Modelo")
        if arquivo:
            arquivo_sol.set(arquivo)
    def selecionar_base():
        arquivo = filedialog.askopenfilename(title="Selecione a Base de Dados")
        if arquivo:
            arquivo_base.set(arquivo)
        
    

    # Crio uma legenda para ficar ao lado do botão.
    lbl_sme = tk.Label(frame, text="Selecione a planilha do SME")
    # Defino a posição do texto na janela.
    lbl_sme.grid(row=0, column=0, pady=5, sticky='w')
    # Crio o botão para salvar o arquivo do SME.
    btn_selecionar_sme = tk.Button(frame, textvariable=arquivo_sme, command=selecionar_sme, wraplength=250, width=40)
    # Defino a posição do botão na janela.
    btn_selecionar_sme.grid(row=0, column=1, padx=5, pady=5)

    # As linhas a seguir são análogas.
    lbl_sma = tk.Label(frame, text="Selecione a planilha do SMA")
    lbl_sma.grid(row=1, column=0, pady=5, sticky='w')
    btn_selecionar_sma = tk.Button(frame, textvariable=arquivo_sma, command=selecionar_sma, wraplength=250, width=40)
    btn_selecionar_sma.grid(row=1, column=1, padx=5, pady=5)

    lbl_scc = tk.Label(frame, text="Selecione a planilha do SCC")
    lbl_scc.grid(row=2, column=0, pady=5, sticky='w')
    btn_selecionar_scc = tk.Button(frame, textvariable=arquivo_scc, command=selecionar_scc, wraplength=250, width=40)
    btn_selecionar_scc.grid(row=2, column=1, padx=5, pady=5)

    lbl_ssc = tk.Label(frame, text="Selecione a planilha do SSC")
    lbl_ssc.grid(row=3, column=0, pady=5, sticky='w')
    btn_selecionar_ssc = tk.Button(frame, textvariable=arquivo_ssc, command=selecionar_ssc, wraplength=250, width=40)
    btn_selecionar_ssc.grid(row=3, column=1, padx=5, pady=5)

    lbl_outros = tk.Label(frame, text="Selecione a planilha dos Outros Institutos")
    lbl_outros.grid(row=4, column=0, pady=5, sticky='w')
    btn_selecionar_outros = tk.Button(frame, textvariable=arquivo_outros, command=selecionar_outros, wraplength=250, width=40)
    btn_selecionar_outros.grid(row=4, column=1, padx=5, pady=5)
    
    lbl_sol = tk.Label(frame, text="Selecione a planilha com os Dados da Solução do Modelo")
    lbl_sol.grid(row=5, column=0, pady=5, sticky='w')
    btn_selecionar_sol = tk.Button(frame, textvariable=arquivo_sol, command=selecionar_sol, wraplength=250, width=40)
    btn_selecionar_sol.grid(row=5, column=1, padx=5, pady=5)

    lbl_base = tk.Label(frame, text="Selecione a Base de Dados")
    lbl_base.grid(row=6, column=0, pady=5, sticky='w')
    btn_selecionar_base = tk.Button(frame, textvariable=arquivo_base, command=selecionar_base, wraplength=250, width=40)
    btn_selecionar_base.grid(row=6, column=1, padx=5, pady=5)

    # Defino uma função que salva os valores das variáveis contendo o nome dos arquivos escolhidos.
    def salvar_valores():
        # Todas as condições a seguir seguem a lógica de que, se um arquivo não foi selecionado, uma janela avisando o ocorrido
        # aparece, pedindo para o usuário selecionar um arquivo no campo requerido.
        
        if not arquivo_sme.get() or arquivo_sme.get() == "Selecione a planilha do SME":
            messagebox.showwarning("Aviso", "Por favor, selecione a planilha do SME.")
            return
        if not arquivo_sma.get() or arquivo_sma.get() == "Selecione a planilha do SMA":
            messagebox.showwarning("Aviso", "Por favor, selecione a planilha do SMA.")
            return
        if not arquivo_scc.get() or arquivo_scc.get() == "Selecione a planilha do SCC":
            messagebox.showwarning("Aviso", "Por favor, selecione a planilha do SCC.")
            return
        if not arquivo_ssc.get() or arquivo_ssc.get() == "Selecione a planilha do SSC":
            messagebox.showwarning("Aviso", "Por favor, selecione a planilha do SSC.")
            return
        if not arquivo_outros.get() or arquivo_outros.get() == "Selecione a planilha dos Outros Institutos":
            messagebox.showwarning("Aviso", "Por favor, selecione a planilha dos Outros Institutos.")
            return
        if not arquivo_sol.get() or arquivo_sol.get() == "Selecione a planilha com os Dados da Solução do Modelo":
            messagebox.showwarning("Aviso", "Por favor, selecione a planilha com os Dados da Solução do Modelo.")
            return
        if not arquivo_base.get() or arquivo_base.get() == "Selecione a Base de Dados":
            messagebox.showwarning("Aviso", "Por favor, selecione a Base de Dados.")
            return

        # Se todos os arquivos tiverem sido selecionados corretamente, crio uma lista com os elencos das disciplinas.
        elenco = [arquivo_sme.get(), arquivo_sma.get(), arquivo_scc.get(), arquivo_ssc.get(), arquivo_outros.get()]

        # E chamo a função que irá realizar o preenchimento de cada um dos arquivos.
        preenchimento(elenco, arquivo_sol.get(), arquivo_base.get(), True)

        # Com a conclusão do preenchimento, aviso o usuário dos novos arquivos preenchidos.
        messagebox.showinfo("Sucesso!", f"Os seguintes arquivos foram criados utilizando os Dados da Solução do Modelo:\n\
        - {arquivo_sme.get().replace('.xlsx', ' Preenchido.xlsx')}\n\
        - {arquivo_sma.get().replace('.xlsx', ' Preenchido.xlsx')}\n\
        - {arquivo_scc.get().replace('.xlsx', ' Preenchido.xlsx')}\n\
        - {arquivo_ssc.get().replace('.xlsx', ' Preenchido.xlsx')}\n\
        - {arquivo_outros.get().replace('.xlsx', ' Preenchido.xlsx')}\n\
        - {arquivo_base.get().replace('.xlsx', ' Preenchido.xlsx')}\n")

        # E fecho a janela que havia sido criada.
        nova_janela.destroy()

    # Defino uma função que salva os valores das variáveis contendo o nome dos arquivos escolhidos.
    def salvar_valores1():
        # Todas as condições a seguir seguem a lógica de que, se um arquivo não foi selecionado, uma janela avisando o ocorrido
        # aparece, pedindo para o usuário selecionar um arquivo no campo requerido.
        
        if not arquivo_sme.get() or arquivo_sme.get() == "Selecione a planilha do SME":
            messagebox.showwarning("Aviso", "Por favor, selecione a planilha do SME.")
            return
        if not arquivo_sma.get() or arquivo_sma.get() == "Selecione a planilha do SMA":
            messagebox.showwarning("Aviso", "Por favor, selecione a planilha do SMA.")
            return
        if not arquivo_scc.get() or arquivo_scc.get() == "Selecione a planilha do SCC":
            messagebox.showwarning("Aviso", "Por favor, selecione a planilha do SCC.")
            return
        if not arquivo_ssc.get() or arquivo_ssc.get() == "Selecione a planilha do SSC":
            messagebox.showwarning("Aviso", "Por favor, selecione a planilha do SSC.")
            return
        if not arquivo_outros.get() or arquivo_outros.get() == "Selecione a planilha dos Outros Institutos":
            messagebox.showwarning("Aviso", "Por favor, selecione a planilha dos Outros Institutos.")
            return
        if not arquivo_sol.get() or arquivo_sol.get() == "Selecione a planilha com os Dados da Solução do Modelo":
            messagebox.showwarning("Aviso", "Por favor, selecione a planilha com os Dados da Solução do Modelo.")
            return
        if not arquivo_base.get() or arquivo_base.get() == "Selecione a Base de Dados":
            messagebox.showwarning("Aviso", "Por favor, selecione a Base de Dados.")
            return
            

        
        # Se todos os arquivos tiverem sido selecionados corretamente, crio uma lista com os elencos das disciplinas.
        elenco = [arquivo_sme.get(), arquivo_sma.get(), arquivo_scc.get(), arquivo_ssc.get(), arquivo_outros.get()]
        
        escolhas_preenchimento(elenco, arquivo_sol.get(), arquivo_base.get())
        
        # nova_janela.wait_window()
        
        # # E chamo a função que irá realizar o preenchimento de cada um dos arquivos.
        # preenchimento(elenco, arquivo_sol.get(), arquivo_base.get())

        # # Com a conclusão do preenchimento, aviso o usuário dos novos arquivos preenchidos.
        # messagebox.showinfo("Sucesso!", f"Os seguintes arquivos foram criados utilizando os Dados da Solução do Modelo:\n\
        # - {arquivo_sme.get().replace('.xlsx', ' Preenchido.xlsx')}\n\
        # - {arquivo_sma.get().replace('.xlsx', ' Preenchido.xlsx')}\n\
        # - {arquivo_scc.get().replace('.xlsx', ' Preenchido.xlsx')}\n\
        # - {arquivo_ssc.get().replace('.xlsx', ' Preenchido.xlsx')}\n\
        # - {arquivo_outros.get().replace('.xlsx', ' Preenchido.xlsx')}\n\
        # - {arquivo_base.get().replace('.xlsx', ' Preenchido.xlsx')}\n")

        # E fecho a janela que havia sido criada.
        # nova_janela.destroy()

    # Defino um botão e sua posição na janela para chamar a função que salva os nomes dos arquivos.
    # btn_salvar = tk.Button(nova_janela, text="Preencher Planilhas com a Solução Completa", command=salvar_valores)
    # btn_salvar.
    btn_salvar1 = ttk.Button(frame, text="Preencher Planilhas com escolhas do usuário", command=salvar_valores1)
    btn_salvar1.grid(row=7, column=0, pady=10)
    
    btn_salvar2 = ttk.Button(frame, text="Preencher Planilhas com a Solução Completa", command=salvar_valores)
    btn_salvar2.grid(row=7, column=1, pady=10)
    
    # Aguarda a interação do usuário antes de continuar a execução
    # nova_janela.wait_window()
    
    # messagebox.showinfo("", f"Arquivo {nome_arquivo} criado com sucesso!")
    # base.to_excel(file_path1.replace('.xlsx', ' Preenchido.xlsx'))
    


## Escolhas de Preenchimento

In [1602]:
# Função que define uma janela para o menu de preenchimento dos arquivos.
def escolhas_preenchimento(elenco1, file_path_sol1, file_path_base1):
    
    elenco = elenco1

    file_path_sol = file_path_sol1

    file_path_base = file_path_base1
    
    # Crio uma nova janela em cima da janela principal da interface.
    nova_janela1 = tk.Toplevel(root)
    nova_janela1.title("Selecione quais aulas devem ser fixadas")
    nova_janela1.geometry("+250+150")
    
    
    
    # Criando um Frame para conter o Canvas e a Scrollbar
    frame_principal = tk.Frame(nova_janela1)
    frame_principal.pack(fill=tk.BOTH, expand=True)
    
    # Criando um Canvas dentro do Frame principal
    canvas = tk.Canvas(frame_principal)
    canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
    
    # Criando uma Scrollbar para o Canvas
    scrollbar = tk.Scrollbar(frame_principal, orient=tk.VERTICAL, command=canvas.yview)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
    
    # Criando um Frame dentro do Canvas para colocar os checkboxes
    frame_checkboxes = tk.Frame(canvas)
    
    # Adicionando o Frame ao Canvas
    canvas.create_window((0, 0), window=frame_checkboxes, anchor="nw")
    
    df = pd.read_excel(file_path_sol)
    # df = pd.read_excel('C:/Users/gabri/Estágio/Códigos/Endgame/Dados da Solução do Modelo.xlsx')
    
    labels = ['Disciplina', 'Horário', 'Sala', 'Inscritos', 'Fixar sala']
    for label in labels:
        tk.Label(frame_checkboxes, text=label).grid(row=0, column=labels.index(label), pady=5, padx=5, sticky='ew')
    ttk.Separator(frame_checkboxes, orient="horizontal").grid(row=1, column=0, columnspan=5, sticky="ew", pady=(0, 10))
    
    df = df.sort_values(by=['Disciplina'], ignore_index=True)
    
    
    vars_checkboxes = []  # Lista para armazenar as variáveis dos checkboxes
    for d in df.index:
        tk.Label(frame_checkboxes, text=df.loc[d, 'Disciplina']).grid(row=2+(2*d), column=0, pady=5, padx=5, sticky='ew')
        tk.Label(frame_checkboxes, text=df.loc[d, 'Horário']).grid(row=2+(2*d), column=1, pady=5, padx=5, sticky='ew')
        tk.Label(frame_checkboxes, text=df.loc[d, 'Sala']).grid(row=2+(2*d), column=2, pady=5, padx=5, sticky='ew')
        tk.Label(frame_checkboxes, text=df.loc[d, 'Inscritos']).grid(row=2+(2*d), column=3, pady=5, padx=5, sticky='ew')
        # tk.Label(frame_checkboxes, text='Fixar').grid(row=d+2, column=4, pady=5, padx=5, sticky='ew')
        ttk.Separator(frame_checkboxes, orient="horizontal").grid(row=3+(2*d), column=0, columnspan=5, sticky="ew", pady=(0, 10))
        var = tk.BooleanVar()
        # checkbox = tk.Checkbutton(frame_checkboxes, text=opcao, variable=var)
        tk.Checkbutton(frame_checkboxes, text='Fixar', variable=var).grid(row=2+(2*d), column=4, pady=5, padx=5, sticky='ew')
        vars_checkboxes.append(var)  # Salvando a variável para futura referência


    

    
            
    # Lista de opções para os Checkboxes
    # opcoes = [f"Fixar {i}" for i in range(1, 31)]  # 30 checkboxes como exemplo
    # vars_checkboxes = []  # Lista para armazenar as variáveis dos checkboxes
    
    # # Criando e adicionando os Checkboxes ao Frame
    # for opcao in opcoes:
    #     var = tk.BooleanVar()
    #     checkbox = tk.Checkbutton(frame_checkboxes, text=opcao, variable=var)
    #     checkbox.pack(anchor="w")
    #     vars_checkboxes.append(var)  # Salvando a variável para futura referência
    
    # Atualizando o tamanho do Frame dentro do Canvas
    def atualizar_scroll(event):
        canvas.configure(scrollregion=canvas.bbox("all"))
    
    frame_checkboxes.bind("<Configure>", atualizar_scroll)
    
    # Configurando a Scrollbar para controlar o Canvas
    canvas.configure(yscrollcommand=scrollbar.set)
    
    
    def salvar_valores():
        
        # print(elenco1, file_path_sol1, file_path_base1)
        elenco = elenco1

        file_path_sol = file_path_sol1
    
        file_path_base = file_path_base1
        # print(elenco, file_path_sol, file_path_base)
        new_df = pd.DataFrame(columns=df.columns)
        for var in vars_checkboxes:
            if var.get():
                index = vars_checkboxes.index(var)
                new_df.loc[len(new_df)] = df.loc[index]

        
        
        # full_name = file_path_sol.replace(".xlsx","com Fixadas.xlsx")
        # i = 1
        # while os.path.exists(full_name):
        #     full_name = f"{file_path_sol} ({i}){'.xlsx'}"
        #     i += 1
        # file_path_sol = full_name
        
        new_df.to_excel(file_path_sol.replace('.xlsx',' com Fixadas.xlsx'), sheet_name="Resultados", index=False)
        
        # Se todos os arquivos tiverem sido selecionados corretamente, crio uma lista com os elencos das disciplinas.
        # elenco = [arquivo_sme.get(), arquivo_sma.get(), arquivo_scc.get(), arquivo_ssc.get(), arquivo_outros.get()]

        # E chamo a função que irá realizar o preenchimento de cada um dos arquivos.
        preenchimento(elenco, file_path_sol.replace('.xlsx',' com Fixadas.xlsx'), file_path_base, False)
        
        # Com a conclusão do preenchimento, aviso o usuário dos novos arquivos preenchidos.
        messagebox.showinfo("Sucesso!", f"O seguinte arquivo foi criado utilizando os Dados da Solução do Modelo:\n\
        - {file_path_sol.replace('.xlsx', ' com Fixadas.xlsx')}\n\
        - {file_path_base.replace('.xlsx', ' Preenchido.xlsx')}\n")

        
        # E fecho a janela que havia sido criada.
        nova_janela1.destroy()

    
    # Determinar a última linha utilizada
    ultima_linha = 2 + 2 * len(df)
    
    ttk.Button(frame_checkboxes, text="Preencher planilhas com salas fixadas", 
               command=salvar_valores).grid(row=ultima_linha, column=3, pady=5, padx=5, sticky='ew')
    
    nova_janela1.wait_window()

## Função para preencher

In [1603]:
# Função que preenche os arquivos fornecidos pelo usuário.
def preenchimento(lista_elenco, file_path_sol, file_path_base, preencher_elenco):
    # Abro a solução dada pelo modelo como um dataframe.
    solucao = pd.read_excel(file_path_sol)
    # df = pd.read_excel('C:/Users/gabri/Estágio/Códigos/Endgame/Dados da solução do Modelo.xlsx')
    # file_path1 = 'C:/Users/gabri/Estágio/Códigos/Endgame/Newest completa (1).xlsx'

    if preencher_elenco:
        # Para cada elenco na lista de elencos:
        for file_path_elenco in lista_elenco:
            # Leio e salvo o arquivo em uma variável.
            elenco = pd.read_excel(file_path_elenco)
            # file_path = 'C:/Users/gabri/Estágio/Códigos/Endgame/Testes/Elenco SME_2025 testando.xlsx'
            # sme = pd.read_excel(file_path)
    
            # Defino uma variável com o nome de um cabeçalho para ser encontrado, caso o cabeçalho não seja a primeira linha da planilha.
            header_name = 'Disciplina (código)'
    
            # Para cada linha e célula da primeira coluna do dataframe:
            for i, valor in enumerate(elenco.loc[:,elenco.columns[0]]):
                # Se o valor da célula for o nome do cabeçalho que estou procurando:
                if valor == header_name:
                    # print(f"Dataframe do {name} lido com sucesso.")
                    # Salvo o número da linha do cabeçalho.
                    header_row = i+1
    
                    # E interrompo o loop.
                    break
    
            # Leio o arquivo da forma correta.
            elenco = pd.read_excel(file_path_elenco, header=header_row)
    
            # Para cada coluna da planilha:
            for col in range(len(elenco.columns)):
                # Verifico se tem um "\n" no texto do dataframe:
                if "\n" in elenco.columns[col]:
                    # Se houver, substituo ele por um " ".
                    elenco = elenco.rename(columns={elenco.columns[col] : elenco.columns[col].replace("\n", " ")})
                # Verifico se estou na coluna das salas:
                if "Sala" in elenco.columns[col]:
                    # Se estiver, substituo o nome da coluna por apenas "Sala"
                    elenco = elenco.rename(columns={elenco.columns[col] : "Sala"})
                        
            # print(sme['Disciplina (código)'].get_loc('SME0104'))
    
            # Como ler o arquivo como um dataframe e salvá-lo altera a estrutura da planilha, preciso abrir o arquivo como um
            # workbook, e utilizarei o dataframe definido anteriormente para utilizar métodos de busca mais eficientes.
            wb = load_workbook(file_path_elenco)
            ws = wb.active
    
            # Para cada linha do dataframe:
            for d in range(len(elenco)):
                # Garanto que o código de uma disciplina não possui espaços.
                elenco.loc[d, 'Disciplina (código)'] = str(elenco.loc[d, 'Disciplina (código)']).replace(' ', '')
                
                # Adiciono um traço '-' e o número da turma ao nome da disciplina para ser igual
                # ao utilizado no modelo.
                disciplina = str(elenco.loc[d, 'Disciplina (código)']) + '-' + str(elenco.loc[d, 'Turma'])
    
                # Verifico se a disciplina em questão foi incluída na solução:
                if disciplina in solucao['Disciplina'].tolist():
                    # print(sme.loc[d, 'Disciplina (código)'])
                    # Se foi, salvo a linha da disciplina com um acréscimo de duas unidades; uma para pular a linha do cabeçalho, e a outra para
                    # igualar com os índices das linhas do workbook.
                    row = d+2
    
                    # Semelhantemente, salvo o índice da coluna de salas do dataframe com um acréscimo de 1.
                    coluna = elenco.columns.get_loc('Sala')+1
    
                    # Limpo o valor da célula da sala da disciplina d, utilizando uma soma de índices para garantir
                    # que a linha está correta.
                    ws.cell(row=header_row+row, column=coluna).value = None
    
                    # Agora, faço um pequeno filtro das linhas de solução da disciplina, ou seja,
                    # filtro os dados da solução para analisar apenas os que pertencem às aulas da disciplina.
                    solucao_filtrada = solucao[solucao['Disciplina'] == disciplina]
                    # print(dfv)
    
                    # Antes de explicar cada linha do trecho seguinte, devo explicar a lógica por trás dele.
                    # Com a solução filtrada, eu tenho, separadamente, os horários das aulas de uma disciplina e a sala
                    # onde cada uma foi alocada. Dessa forma, eu consigo iterar no comprimento da solução filtrada,
                    # que deve me dar o número de colunas de horário que aquela disciplina possui.
                    # E se eu cruzar essa iteração com uma que verifica se uma das aulas corresponde com a atual coluna de horário,
                    # eu consigo cruzar colocar, em ordem, qual a sala de cada aula.
                    
                    # Para cada linha dessa solução filtrada, que diz respeito a cada aula da disciplina:
                    # for i in range(len(solucao_filtrada)):
                    # novo_valor = ['0', '0', '0', '0']
                    for i in range(len(solucao_filtrada)):
                        # E para cada aula da disciplina na solução filtrada:
                        for a in solucao_filtrada.index:
                            # print(sme.loc[d, 'Horário ' + str(i+1)])
                            # print(dfv.loc[a, 'Horário'])
                            # Verifico se o horário da coluna 'Horário i+1' bate com o horário da aula 'a':
                            if elenco.loc[d, 'Horário ' + str(i+1)] == solucao_filtrada.loc[a, 'Horário']:
                                # novo_valor[i] = str(solucao_filtrada.loc[a, 'Sala'])
                                # Se os horários batem, verifico se algum outro horário já foi colocado na célula:
                                if ws.cell(row=header_row+row, column=coluna).value:
                                    # Se foi, defino o novo valor da célula como o valor antigo somado com o novo, mas separado por uma vírgula.
                                    novo_valor = str(ws.cell(row=header_row+row, column=coluna).value) + ', ' + str(solucao_filtrada.loc[a, 'Sala'])
    
                                    # E defino o valor da célula com esse novo valor.
                                    ws.cell(row=header_row+row, column=coluna).value = novo_valor
    
                                # Como eu havia limpado todas as células de sala anteriormente, tenho certeza que não há nenhum outro horário,
                                # então coloco o primeiro na célula respectiva.
                                else:
                                    ws.cell(row=header_row+row, column=coluna).value = str(solucao_filtrada.loc[a, 'Sala'])
                    # salas_fixadas = novo_valor[0]
                    # for s in novo_valor[1:]:
                    #     salas_fixadas += ', ' + str(s)
                        
                    # ws.cell(row=header_row+row, column=coluna).value = salas_fixadas
    
            # Após isso, salvo um novo arquivo de elenco de mesmo nome, mas adicionando 'Preenchido' para saber qual é qual.
            wb.save(file_path_elenco.replace('.xlsx', ' Preenchido.xlsx'))


    # Com todos os elencos preenchidos, agora resta preencher a base de dados para o modelo.
    # Assim, abro a base de dados como um arquivo Excel.
    base = pd.ExcelFile(file_path_base)

    # Salvo quais as planilhas do arquivo.
    sheet_names = base.sheet_names

    # E leio novamente o arquivo, mas como um dataframe com as planilhas corretas.
    base = pd.read_excel(file_path_base, sheet_name=sheet_names)
    
    # sheet_names = aux.sheet_names
    # base = pd.read_excel(file_path1, sheet_name=sheet_names)

    # Para cada planilha de disciplinas, isto é, para cada planilha após a planilha de salas:
    for sheet in sheet_names[1:]:
        # Refaço a mesma lógica feita anteriormente. Verificarei se a disciplina foi alocada em alguma sala, filtrarei o dataframe,
        # e cruzarei os dados da solução com os do dataframe, deixando o mesmo formato e ordem das salas para cada aula das disciplinas.
        for d in range(len(base[sheet])):
            disciplina = base[sheet].loc[d, 'Disciplina (código)']

            if disciplina in solucao['Disciplina'].tolist():
                base[sheet].loc[d, 'Sala'] = None
                solucao_filtrada = solucao[solucao['Disciplina'] == disciplina]
                
                # for i in range(len(solucao_filtrada)):
                novo_valor = ['0', '0', '0', '0']
                for i in range(4):
                    for a in solucao_filtrada.index:
                        if base[sheet].loc[d, 'Horário ' + str(i+1)] == solucao_filtrada.loc[a, 'Horário']:
                            novo_valor[i] = str(solucao_filtrada.loc[a, 'Sala'])
                            # if not pd.isna(base[sheet].loc[d, 'Sala']):
                            #     # novo_valor += str(solucao_filtrada.loc[a, 'Sala'])
                            #     novo_valor = str(base[sheet].loc[d, 'Sala']) + ', ' + str(solucao_filtrada.loc[a, 'Sala'])
                            #     base[sheet].loc[d, 'Sala'] = novo_valor
                            # else:
                            #     base[sheet].loc[d, 'Sala'] = str(solucao_filtrada.loc[a, 'Sala'])
                                # novo_valor = str(solucao_filtrada.loc[a, 'Sala'])
                salas_fixadas = novo_valor[0]
                for s in novo_valor[1:]:
                    salas_fixadas += ', ' + str(s)
                    
                base[sheet].loc[d, 'Sala'] = salas_fixadas

    # Com o novo dataframe construído, salvo-o como arquivo Excel com o nome alterado para distinção.
    with pd.ExcelWriter(file_path_base.replace('.xlsx', ' Preenchido.xlsx'), engine="openpyxl") as writer:
        for sheet in sheet_names:
            base[sheet].to_excel(writer, sheet_name=sheet, index=False)

    

# Interface

In [1604]:
# Aqui é onde a janela principal da interface (root) é montada.
# Defino ela com a variável root, e dou-lhe um título.
root = tk.Tk()
root.title("Menu Principal")

# Após isso, adiciono um frame para a janela.
frm = ttk.Frame(root, padding=10)
frm.grid()

# E adiciono os botões que chamam as funções definidas até então.
# Como eu não preciso que os botões salvem algo aqui, não é necessário adicioná-los como variáveis com nome.
# Vale também comentar que algumas funções chamadas pelos botões possuem parâmetros, e para passá-los, é necessário
# usar a "função lambda".
ttk.Button(frm, text="Construir Planilha com os dados das Aulas", command=lambda: planilha_dep(jupiter=False)).grid(column=0, row=0, sticky="w", pady=5)
ttk.Button(frm, text="Construir Planilha com os dados do JúpiterWeb", command=lambda: planilha_dep(jupiter=True)).grid(column=0, row=1, sticky="w", pady=5)
ttk.Button(frm, text="Construir Base de Dados do Modelo", command=lambda: base_dados(pior_caso=False)).grid(column=0,row=2,sticky="w", pady=5)
ttk.Button(frm, text="Construir Base de Dados de Pior Caso", command=lambda: base_dados(pior_caso=True)).grid(column=0,row=3,sticky="w", pady=5)
ttk.Button(frm, text="Verificar Dados e Executar Modelo", command=execute).grid(column=0,row=4,sticky="w", pady=5)
ttk.Button(frm, text="Relatório de Espaços Livres", command=analise_vazios).grid(column=0,row=5,sticky="w",pady=5)
ttk.Button(frm, text="Preencher planilhas com Dados da Solução", command=preencher_planilha_dados).grid(column=0, row=6, sticky="w", pady=5)
ttk.Button(frm, text="Sair", command=root.destroy).grid(column=0, row=7, sticky="w", pady=5)
ttk.Button(frm, text="Teste", command=lambda: Novo_edit_config(file_name='C:/Users/gabri/Estágio/Códigos/Endgame/Newest completa (1).xlsx')).grid(column=0, row=8, sticky="w", pady=5)

# Com ela definida, basta colocá-la para rodar.
root.mainloop()

# print(type(df))

    Disciplina                                              Nomes  \
0    5500004-1                  Fundamentos de Ciências de Dados    
1    5500004-1                  Fundamentos de Ciências de Dados    
2    5500005-1       Informação Profissional em Ciência de Dados    
3    7600005-3                                           Física 1   
4    7600005-3                                           Física 1   
..         ...                                                ...   
236  SSC0960-2                              Programação Funcional   
237  SSC0960-3                              Programação Funcional   
238  SSC0961-1  Desenvolvimento Web e Mobile (CEE Engenharia d...   
239  SSC0963-1             Indústria 4.0: Tecnologia e Aplicações   
240  SSC0964-1     Introcução  à Computação no Mercado Financeiro   

                   Cursos                Horário         Sala  Inscritos  \
0                 BCDados    Terça - 10:10/11:50        3-011         35   
1                 B

# Testes

In [1480]:
proibir_sala = {}
proibir_sala.update({'a': 1})
proibir_sala.update({'b': 2})
proibir_sala.update({'c': 3})
proibir_sala.update({'d': 4})

print(proibir_sala.keys())
if 'a' in proibir_sala.keys():
    value = proibir_sala.get('a')
    print(value)

dict_keys(['a', 'b', 'c', 'd'])
1


In [341]:
# Criando a janela principal
root = tk.Tk()
root.title("Lista de Checkboxes com Scroll")

# Criando um Frame para conter o Canvas e a Scrollbar
frame_principal = tk.Frame(root)
frame_principal.pack(fill=tk.BOTH, expand=True)

# Criando um Canvas dentro do Frame principal
canvas = tk.Canvas(frame_principal)
canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

# Criando uma Scrollbar para o Canvas
scrollbar = tk.Scrollbar(frame_principal, orient=tk.VERTICAL, command=canvas.yview)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

# Criando um Frame dentro do Canvas para colocar os checkboxes
frame_checkboxes = tk.Frame(canvas)

# Adicionando o Frame ao Canvas
canvas.create_window((0, 0), window=frame_checkboxes, anchor="nw")

# Lista de opções para os Checkboxes
opcoes = [f"Opção {i}" for i in range(1, 31)]  # 30 checkboxes como exemplo
vars_checkboxes = []  # Lista para armazenar as variáveis dos checkboxes

# Criando e adicionando os Checkboxes ao Frame
for opcao in opcoes:
    var = tk.BooleanVar()
    checkbox = tk.Checkbutton(frame_checkboxes, text=opcao, variable=var)
    checkbox.pack(anchor="w")
    vars_checkboxes.append(var)  # Salvando a variável para futura referência

# Atualizando o tamanho do Frame dentro do Canvas
def atualizar_scroll(event):
    canvas.configure(scrollregion=canvas.bbox("all"))

frame_checkboxes.bind("<Configure>", atualizar_scroll)

# Configurando a Scrollbar para controlar o Canvas
canvas.configure(yscrollcommand=scrollbar.set)

# Rodando o loop principal
root.mainloop()